# DEF. MODELO ML (XGBoost - OFFER)

## MODULES & LIBRARIES

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#conda update scikit-learn

In [3]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import datetime as dt
#import pandas_profiling
from math import *

import sklearn
import matplotlib
import scipy
import statsmodels
import mlxtend
import category_encoders
import seaborn
import xgboost

In [5]:
# IMPORT MODULES

# Base -----------------------------------------------------------
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime

# Viz ------------------------------------------------------------
import matplotlib.pyplot as plt
import matplotlib.colors as ma
import matplotlib.font_manager as fm
import seaborn as sns
import plotly
import plotnine
from plotnine import *
#import patchworklib as pw

%matplotlib inline

In [6]:
import time
from datetime import timedelta

In [7]:
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OrdinalEncoder
from category_encoders import OrdinalEncoder

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

In [9]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.5 MB/s eta 0:00:00


In [10]:
import optuna

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
%matplotlib inline

In [12]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

## 1. PREPARACIÓN DE DATOS

#### - Data (BID/OFFER MODEL)

In [13]:
# CARGA DATOS
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. SP).csv'
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP).csv'
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 60SP).csv'
#file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP+GSP).csv'
file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP+GSP).csv'
df_model = pd.read_csv(file_path)

In [14]:
cat_vars = list(df_model.select_dtypes(exclude=["number"]).columns)
num_vars = list(df_model.select_dtypes(include=["number"]).columns)

In [15]:
print(f'CATEGORICAL VARIABLES ({len(cat_vars)})')
print('-'*30)
i = 0

for x in cat_vars:
      i += 1
      var_values = list(df_model[x].unique())
      if len(var_values)>15:
        ex_values = str(var_values[:15]) + '...'
      else:
        ex_values = var_values

      print(f'{i}. "{x}": {len(var_values)} unique values ({ex_values})')

CATEGORICAL VARIABLES (14)
------------------------------
1. "recordtype": 2 unique values (['BID', 'OFFER'])
2. "settlementDate": 910 unique values (['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15']...)
3. "BMU ID": 483 unique values (['E_GYAR-1', 'E_SHOS-1', 'T_CDCL-1', 'T_MRWD-1', 'T_PEMB-11', 'T_SCCL-3', 'T_SHBA-1', 'T_CARR-2', 'T_DAMC-1', 'T_GRAI-6', 'T_GRAI-7', 'T_GRAI-8', 'T_MEDP-1', 'T_SEAB-1', 'T_PEMB-31']...)
4. "BMU Type": 6 unique values (['E', 'T', '2', 'V', 'M', 'C'])
5. "BMU Fuel Type": 9 unique values (['CCGT', 'PS', 'COAL', 'OTHER', 'NPSHYD', 'OCGT', 'BATTERY', 'BIOMASS', 'WIND'])
6. "BMU GSP Group Name": 14 unique values (['Eastern England', 'South Eastern England', 'East Midlands', 'Southern England', 'Southern Wales', 'Yorkshire', 'North Western England', 'South Western England', 'Northern Scotland', 'Mersey

In [16]:
df_model = df_model.drop(columns=['BMU ID', 'BMU Party ID'], axis=1)

In [17]:
# DATE COLUMN
df_model['settlementDate'] = pd.to_datetime(df_model['settlementDate'])


# Transforming date to CATEGORICAL ORDINAL
order = list(df_model['settlementDate'].unique())
order.sort()
print(order)
data = df_model['settlementDate']
df_model['settlementDate'] = pd.Categorical(data, order, ordered=True)


[numpy.datetime64('2021-01-01T00:00:00.000000000'), numpy.datetime64('2021-01-02T00:00:00.000000000'), numpy.datetime64('2021-01-03T00:00:00.000000000'), numpy.datetime64('2021-01-04T00:00:00.000000000'), numpy.datetime64('2021-01-05T00:00:00.000000000'), numpy.datetime64('2021-01-06T00:00:00.000000000'), numpy.datetime64('2021-01-07T00:00:00.000000000'), numpy.datetime64('2021-01-08T00:00:00.000000000'), numpy.datetime64('2021-01-09T00:00:00.000000000'), numpy.datetime64('2021-01-10T00:00:00.000000000'), numpy.datetime64('2021-01-11T00:00:00.000000000'), numpy.datetime64('2021-01-12T00:00:00.000000000'), numpy.datetime64('2021-01-13T00:00:00.000000000'), numpy.datetime64('2021-01-14T00:00:00.000000000'), numpy.datetime64('2021-01-15T00:00:00.000000000'), numpy.datetime64('2021-01-16T00:00:00.000000000'), numpy.datetime64('2021-01-17T00:00:00.000000000'), numpy.datetime64('2021-01-18T00:00:00.000000000'), numpy.datetime64('2021-01-19T00:00:00.000000000'), numpy.datetime64('2021-01-20T0

In [18]:
df_model['settlementDate'].unique()

[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-04, 2021-01-05, ..., 2023-06-26, 2023-06-27, 2023-06-28, 2023-06-29, 2023-06-30]
Length: 910
Categories (910, datetime64[ns]): [2021-01-01 < 2021-01-02 < 2021-01-03 < 2021-01-04 ... 2023-06-27 <
                                   2023-06-28 < 2023-06-29 < 2023-06-30]

In [19]:
# CATEGORICAL ORDINAL

# Column "settlementPeriod"
order = list(df_model['settlementPeriod'].astype('int').unique())
order.sort()
print(order)
data = df_model['settlementPeriod']
df_model['settlementPeriod'] = pd.Categorical(data, order, ordered=True)

# Column "Year"
order = list(df_model['Year'].astype('int').unique())
order.sort()
print(order)
data = df_model['Year']
df_model['Year'] = pd.Categorical(data, order, ordered=True)

# Column "Month"
order = list(df_model['Month'].astype('int').unique())
order.sort()
print(order)
data = df_model['Month']
df_model['Month'] = pd.Categorical(data, order, ordered=True)

# Column "Day"
order = list(df_model['Day'].astype('int').unique())
order.sort()
print(order)
data = df_model['Day']
df_model['Day'] = pd.Categorical(data, order, ordered=True)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
[2021, 2022, 2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [20]:
# CATEGORICAL NOMINAL

# Column "recordtype"
data = df_model['recordtype']
df_model['recordtype'] = pd.Categorical(data, ordered=False)

# Column "BMU ID"
#data = df_model['BMU ID']
#df_model['BMU ID'] = pd.Categorical(data, ordered=False)

# Column "BMU Type"
data = df_model['BMU Type']
df_model['BMU Type'] = pd.Categorical(data, ordered=False)

# Column "BMU Fuel Type"
data = df_model['BMU Fuel Type']
df_model['BMU Fuel Type'] = pd.Categorical(data, ordered=False)

# Column "BMU GSP Group Name"
data = df_model['BMU GSP Group Name']
df_model['BMU GSP Group Name'] = pd.Categorical(data, ordered=False)

# Column "BZONE"
data = df_model['BZONE']
df_model['BZONE'] = pd.Categorical(data, ordered=False)

# Column "BMU Party ID"
#data = df_model['BMU Party ID']
#df_model['BMU Party ID'] = pd.Categorical(data, ordered=False)

# Column "Trading Unit"
data = df_model['Trading Unit']
df_model['Trading Unit'] = pd.Categorical(data, ordered=False)

# Column "PC Flag"
data = df_model['PC Flag']
df_model['PC Flag'] = pd.Categorical(data, ordered=False)

# Column "PC Status"
data = df_model['PC Status']
df_model['PC Status'] = pd.Categorical(data, ordered=False)

# Column "Exempt Export Flag"
data = df_model['Exempt Export Flag']
df_model['Exempt Export Flag'] = pd.Categorical(data, ordered=False)

# Column "Base TU Flag"
data = df_model['Base TU Flag']
df_model['Base TU Flag'] = pd.Categorical(data, ordered=False)

# Column "FPN Flag"
data = df_model['FPN Flag']
df_model['FPN Flag'] = pd.Categorical(data, ordered=False)


In [21]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Columns: 209 entries, recordtype to sp48_totalvolume
dtypes: category(16), float64(191), int64(2)
memory usage: 1.7 GB


In [22]:
df_model['recordtype'].value_counts(normalize=True)

BID      0.625496
OFFER    0.374504
Name: recordtype, dtype: float64

In [23]:
#drop_vars = ['settlementDate', 'settlementPeriod', 'BMU ID', 'BMU GSP Group Name', 'BMU Party ID', 'Year', 'Month', 'Day']

In [24]:
df_model.isnull().sum()

recordtype          0
settlementDate      0
settlementPeriod    0
acceptedprice       0
acceptedvolume      0
                   ..
sp47_meanvolume     0
sp47_totalvolume    0
sp48_pavgprice      0
sp48_meanvolume     0
sp48_totalvolume    0
Length: 209, dtype: int64

In [25]:
df_model = df_model.fillna(value=99999, method=None, axis=None, inplace=False, limit=None, downcast=None)

In [26]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Columns: 209 entries, recordtype to sp48_totalvolume
dtypes: category(16), float64(191), int64(2)
memory usage: 1.7 GB


#### - Arg. LAG Variables (BID/OFFER MODEL)

In [27]:
df_model.iloc[:,68:134].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 66 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   NatGas_price      1187159 non-null  float64
 1   Coal_price        1187159 non-null  float64
 2   BrentOil_price    1187159 non-null  float64
 3   sp3_pavgprice     1187159 non-null  float64
 4   sp3_meanvolume    1187159 non-null  float64
 5   sp3_totalvolume   1187159 non-null  float64
 6   sp4_pavgprice     1187159 non-null  float64
 7   sp4_meanvolume    1187159 non-null  float64
 8   sp4_totalvolume   1187159 non-null  float64
 9   sp5_pavgprice     1187159 non-null  float64
 10  sp5_meanvolume    1187159 non-null  float64
 11  sp5_totalvolume   1187159 non-null  float64
 12  sp6_pavgprice     1187159 non-null  float64
 13  sp6_meanvolume    1187159 non-null  float64
 14  sp6_totalvolume   1187159 non-null  float64
 15  sp7_pavgprice     1187159 non-null  float64
 16  

In [28]:
df_model.iloc[:,134:210].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 75 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   sp24_pavgprice    1187159 non-null  float64
 1   sp24_meanvolume   1187159 non-null  float64
 2   sp24_totalvolume  1187159 non-null  float64
 3   sp25_pavgprice    1187159 non-null  float64
 4   sp25_meanvolume   1187159 non-null  float64
 5   sp25_totalvolume  1187159 non-null  float64
 6   sp26_pavgprice    1187159 non-null  float64
 7   sp26_meanvolume   1187159 non-null  float64
 8   sp26_totalvolume  1187159 non-null  float64
 9   sp27_pavgprice    1187159 non-null  float64
 10  sp27_meanvolume   1187159 non-null  float64
 11  sp27_totalvolume  1187159 non-null  float64
 12  sp28_pavgprice    1187159 non-null  float64
 13  sp28_meanvolume   1187159 non-null  float64
 14  sp28_totalvolume  1187159 non-null  float64
 15  sp29_pavgprice    1187159 non-null  float64
 16  

In [29]:
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

In [30]:
df_aux = pd.DataFrame()

In [31]:
sp_list = list(range(3,7))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_6_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp3_6_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp3_6_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)

In [32]:
sp_list = list(range(6,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp6_12_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp6_12_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp6_12_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)

In [33]:
'''
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_12_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp3_12_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp3_12_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)
'''

"\nsp_list = list(range(3,13))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp3_12_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp3_12_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp3_12_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)\n"

In [34]:
'''
sp_list = list(range(12,24))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp12_24_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp12_24_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp12_24_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)
'''

"\nsp_list = list(range(12,24))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp12_24_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp12_24_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp12_24_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)\n"

In [35]:
'''
sp_list = list(range(24,36))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp24_36_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp24_36_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp24_36_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)
'''

"\nsp_list = list(range(24,36))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp24_36_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp24_36_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp24_36_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)\n"

In [36]:
'''
sp_list = list(range(36,48))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp36_48_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)
df_aux['sp36_48_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)
df_aux['sp36_48_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)
'''

"\nsp_list = list(range(36,48))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp36_48_pavgprice'] = df_model[cols_pavgprice].mean(axis=1)\ndf_aux['sp36_48_meanvolume'] = df_model[cols_meanvolume].mean(axis=1)\ndf_aux['sp36_48_totalvolume'] = df_model[cols_totalvolume].sum(axis=1)\n"

In [37]:
df_aux

,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,22.893185,0.000000,0.000,27.104873,0.000000,0.000
1,37.407064,-30.982417,-451.889,37.889961,-33.565774,-914.094
2,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
3,38.280000,-92.192250,-368.769,37.630000,-89.602571,-627.218
4,35.535761,-23.795667,-230.932,38.187343,-15.829571,-110.807
...,...,...,...,...,...,...
1187154,121.012684,1.780750,7.123,119.610792,2.552571,27.380
1187155,122.405000,0.000000,0.000,117.192857,0.035071,0.491
1187156,122.405000,0.000000,0.000,117.192857,0.035071,0.491
1187157,124.000000,167.500000,670.000,120.117143,91.046429,637.325


In [38]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   sp3_6_pavgprice     1187159 non-null  float64
 1   sp3_6_meanvolume    1187159 non-null  float64
 2   sp3_6_totalvolume   1187159 non-null  float64
 3   sp6_12_pavgprice    1187159 non-null  float64
 4   sp6_12_meanvolume   1187159 non-null  float64
 5   sp6_12_totalvolume  1187159 non-null  float64
dtypes: float64(6)
memory usage: 54.3 MB


In [39]:
drop_cols = [x for x in list(df_model.columns) if x.startswith('sp')==True]
drop_cols[:5]

['sp3_pavgprice',
 'sp3_meanvolume',
 'sp3_totalvolume',
 'sp4_pavgprice',
 'sp4_meanvolume']

In [40]:
df_model = df_model.drop(columns=drop_cols, axis=1)

In [41]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 71 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1187159 non-null  category
 1   settlementDate                 1187159 non-null  category
 2   settlementPeriod               1187159 non-null  category
 3   acceptedprice                  1187159 non-null  float64 
 4   acceptedvolume                 1187159 non-null  float64 
 5   BMU Type                       1187159 non-null  category
 6   BMU Fuel Type                  1187159 non-null  category
 7   BMU GSP Group Name             1187159 non-null  category
 8   LOC LAT                        1187159 non-null  float64 
 9   LOC LONG                       1187159 non-null  float64 
 10  LOC Center LAT                 1187159 non-null  float64 
 11  LOC Center LONG                1187159 non-null  float64 
 12  

In [42]:
df_model = pd.concat([df_model, df_aux], axis=1)

In [43]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187159 entries, 0 to 1187158
Data columns (total 77 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1187159 non-null  category
 1   settlementDate                 1187159 non-null  category
 2   settlementPeriod               1187159 non-null  category
 3   acceptedprice                  1187159 non-null  float64 
 4   acceptedvolume                 1187159 non-null  float64 
 5   BMU Type                       1187159 non-null  category
 6   BMU Fuel Type                  1187159 non-null  category
 7   BMU GSP Group Name             1187159 non-null  category
 8   LOC LAT                        1187159 non-null  float64 
 9   LOC LONG                       1187159 non-null  float64 
 10  LOC Center LAT                 1187159 non-null  float64 
 11  LOC Center LONG                1187159 non-null  float64 
 12  

#### - Data (SP PAVG. PRICE MODEL)

In [44]:
# CARGA DATOS
file_path = '/content/drive/MyDrive/QEnergy/Data/Data Model (Allboas & Research & Prev. 48SP+GSP (pavg. price per SP+GSP)).csv'
df_modelAVG = pd.read_csv(file_path)

In [45]:
cat_vars = list(df_modelAVG.select_dtypes(exclude=["number"]).columns)
num_vars = list(df_modelAVG.select_dtypes(include=["number"]).columns)

In [46]:
print(f'CATEGORICAL VARIABLES ({len(cat_vars)})')
print('-'*30)
i = 0

for x in cat_vars:
      i += 1
      var_values = list(df_modelAVG[x].unique())
      if len(var_values)>15:
        ex_values = str(var_values[:15]) + '...'
      else:
        ex_values = var_values

      print(f'{i}. "{x}": {len(var_values)} unique values ({ex_values})')

CATEGORICAL VARIABLES (3)
------------------------------
1. "recordtype": 2 unique values (['BID', 'OFFER'])
2. "settlementDate": 910 unique values (['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15']...)
3. "BMU GSP Group Name": 14 unique values (['East Midlands', 'Eastern England', 'London', 'Merseyside and Northern Wales', 'North Eastern England', 'North Western England', 'Northern Scotland', 'South Eastern England', 'South Western England', 'Southern England', 'Southern Scotland', 'Southern Wales', 'West Midlands', 'Yorkshire'])


In [47]:
# DATE COLUMN
df_modelAVG['settlementDate'] = pd.to_datetime(df_modelAVG['settlementDate'])


# Transforming date to CATEGORICAL ORDINAL
order = list(df_modelAVG['settlementDate'].unique())
order.sort()
print(order)
data = df_modelAVG['settlementDate']
df_modelAVG['settlementDate'] = pd.Categorical(data, order, ordered=True)


[numpy.datetime64('2021-01-01T00:00:00.000000000'), numpy.datetime64('2021-01-02T00:00:00.000000000'), numpy.datetime64('2021-01-03T00:00:00.000000000'), numpy.datetime64('2021-01-04T00:00:00.000000000'), numpy.datetime64('2021-01-05T00:00:00.000000000'), numpy.datetime64('2021-01-06T00:00:00.000000000'), numpy.datetime64('2021-01-07T00:00:00.000000000'), numpy.datetime64('2021-01-08T00:00:00.000000000'), numpy.datetime64('2021-01-09T00:00:00.000000000'), numpy.datetime64('2021-01-10T00:00:00.000000000'), numpy.datetime64('2021-01-11T00:00:00.000000000'), numpy.datetime64('2021-01-12T00:00:00.000000000'), numpy.datetime64('2021-01-13T00:00:00.000000000'), numpy.datetime64('2021-01-14T00:00:00.000000000'), numpy.datetime64('2021-01-15T00:00:00.000000000'), numpy.datetime64('2021-01-16T00:00:00.000000000'), numpy.datetime64('2021-01-17T00:00:00.000000000'), numpy.datetime64('2021-01-18T00:00:00.000000000'), numpy.datetime64('2021-01-19T00:00:00.000000000'), numpy.datetime64('2021-01-20T0

In [48]:
df_modelAVG['settlementDate'].unique()

[2021-01-01, 2021-01-02, 2021-01-03, 2021-01-04, 2021-01-05, ..., 2023-06-26, 2023-06-27, 2023-06-28, 2023-06-29, 2023-06-30]
Length: 910
Categories (910, datetime64[ns]): [2021-01-01 < 2021-01-02 < 2021-01-03 < 2021-01-04 ... 2023-06-27 <
                                   2023-06-28 < 2023-06-29 < 2023-06-30]

In [49]:
# CATEGORICAL ORDINAL

# Column "settlementPeriod"
order = list(df_modelAVG['settlementPeriod'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['settlementPeriod']
df_modelAVG['settlementPeriod'] = pd.Categorical(data, order, ordered=True)

# Column "Year"
order = list(df_modelAVG['Year'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['Year']
df_modelAVG['Year'] = pd.Categorical(data, order, ordered=True)

# Column "Month"
order = list(df_modelAVG['Month'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['Month']
df_modelAVG['Month'] = pd.Categorical(data, order, ordered=True)

# Column "Day"
order = list(df_modelAVG['Day'].astype('int').unique())
order.sort()
print(order)
data = df_modelAVG['Day']
df_modelAVG['Day'] = pd.Categorical(data, order, ordered=True)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
[2021, 2022, 2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [50]:
# CATEGORICAL NOMINAL

# Column "recordtype"
data = df_modelAVG['recordtype']
df_modelAVG['recordtype'] = pd.Categorical(data, ordered=False)

# Column "BMU GSP Group Name"
data = df_modelAVG['BMU GSP Group Name']
df_modelAVG['BMU GSP Group Name'] = pd.Categorical(data, ordered=False)


In [51]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Columns: 175 entries, recordtype to sp48_totalvolume
dtypes: category(7), float64(168)
memory usage: 604.7 MB


In [52]:
df_modelAVG['recordtype'].value_counts(normalize=True)

BID      0.506824
OFFER    0.493176
Name: recordtype, dtype: float64

In [53]:
#drop_vars = ['settlementDate', 'settlementPeriod', 'BMU ID', 'BMU GSP Group Name', 'BMU Party ID', 'Year', 'Month', 'Day']

In [54]:
df_modelAVG.isnull().sum()

recordtype            0
settlementDate        0
settlementPeriod      0
BMU GSP Group Name    0
sp_pavgprice          0
                     ..
sp47_meanvolume       0
sp47_totalvolume      0
sp48_pavgprice        0
sp48_meanvolume       0
sp48_totalvolume      0
Length: 175, dtype: int64

In [55]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Columns: 175 entries, recordtype to sp48_totalvolume
dtypes: category(7), float64(168)
memory usage: 604.7 MB


#### - Arg. LAG Variables (SP PAVG. PRICE MODEL)

In [56]:
df_modelAVG.iloc[:,36:102].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 66 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   BrentOil_price    468991 non-null  float64
 1   sp3_pavgprice     468991 non-null  float64
 2   sp3_meanvolume    468991 non-null  float64
 3   sp3_totalvolume   468991 non-null  float64
 4   sp4_pavgprice     468991 non-null  float64
 5   sp4_meanvolume    468991 non-null  float64
 6   sp4_totalvolume   468991 non-null  float64
 7   sp5_pavgprice     468991 non-null  float64
 8   sp5_meanvolume    468991 non-null  float64
 9   sp5_totalvolume   468991 non-null  float64
 10  sp6_pavgprice     468991 non-null  float64
 11  sp6_meanvolume    468991 non-null  float64
 12  sp6_totalvolume   468991 non-null  float64
 13  sp7_pavgprice     468991 non-null  float64
 14  sp7_meanvolume    468991 non-null  float64
 15  sp7_totalvolume   468991 non-null  float64
 16  sp8_pavgprice     46

In [57]:
df_modelAVG.iloc[:,102:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 73 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   sp24_totalvolume  468991 non-null  float64
 1   sp25_pavgprice    468991 non-null  float64
 2   sp25_meanvolume   468991 non-null  float64
 3   sp25_totalvolume  468991 non-null  float64
 4   sp26_pavgprice    468991 non-null  float64
 5   sp26_meanvolume   468991 non-null  float64
 6   sp26_totalvolume  468991 non-null  float64
 7   sp27_pavgprice    468991 non-null  float64
 8   sp27_meanvolume   468991 non-null  float64
 9   sp27_totalvolume  468991 non-null  float64
 10  sp28_pavgprice    468991 non-null  float64
 11  sp28_meanvolume   468991 non-null  float64
 12  sp28_totalvolume  468991 non-null  float64
 13  sp29_pavgprice    468991 non-null  float64
 14  sp29_meanvolume   468991 non-null  float64
 15  sp29_totalvolume  468991 non-null  float64
 16  sp30_pavgprice    46

In [58]:
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

In [59]:
df_aux = pd.DataFrame()

In [60]:
sp_list = list(range(3,7))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_6_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp3_6_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp3_6_totalvolume'] = df_modelAVG[cols_totalvolume].sum(axis=1)

In [61]:
sp_list = list(range(6,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp6_12_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp6_12_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp6_12_totalvolume'] = df_modelAVG[cols_totalvolume].sum(axis=1)

In [62]:
'''
sp_list = list(range(3,13))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp3_12_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp3_12_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp3_12_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(3,13))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp3_12_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp3_12_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp3_12_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [63]:
'''
sp_list = list(range(12,24))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp12_24_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp12_24_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp12_24_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(12,24))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp12_24_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp12_24_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp12_24_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [64]:
'''
sp_list = list(range(24,36))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp24_36_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp24_36_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp24_36_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(24,36))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp24_36_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp24_36_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp24_36_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [65]:
'''
sp_list = list(range(36,48))
cols_pavgprice = []
cols_meanvolume = []
cols_totalvolume = []

for x in sp_list:
  cols_pavgprice.append('sp' + str(x) + '_pavgprice')
  cols_meanvolume.append('sp' + str(x) + '_meanvolume')
  cols_totalvolume.append('sp' + str(x) + '_totalvolume')

df_aux['sp36_48_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)
df_aux['sp36_48_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)
df_aux['sp36_48_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)
'''

"\nsp_list = list(range(36,48))\ncols_pavgprice = []\ncols_meanvolume = []\ncols_totalvolume = []\n\nfor x in sp_list:\n  cols_pavgprice.append('sp' + str(x) + '_pavgprice')\n  cols_meanvolume.append('sp' + str(x) + '_meanvolume')\n  cols_totalvolume.append('sp' + str(x) + '_totalvolume')\n\ndf_aux['sp36_48_pavgprice'] = df_modelAVG[cols_pavgprice].mean(axis=1)\ndf_aux['sp36_48_meanvolume'] = df_modelAVG[cols_meanvolume].mean(axis=1)\ndf_aux['sp36_48_totalvolume'] = df_modelAVG[cols_totalvolume].mean(axis=1)\n"

In [66]:
df_aux

,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
1,73.322564,0.002250,0.009,93.948627,0.001286,0.009
2,77.072564,16.968750,67.875,88.109188,0.001286,0.009
3,73.728120,51.656250,206.625,84.133499,0.001286,0.009
4,71.737500,132.302250,529.209,80.957125,0.001286,0.009
...,...,...,...,...,...,...
468986,62.010000,-128.952750,-515.811,54.640423,-85.697333,-817.925
468987,116.460000,0.000000,0.000,118.610792,0.000000,0.000
468988,62.010000,-164.104250,-656.417,61.124296,-102.506905,-926.759
468989,62.005671,-150.664125,-675.396,61.920697,-113.622667,-924.752


In [67]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   sp3_6_pavgprice     468991 non-null  float64
 1   sp3_6_meanvolume    468991 non-null  float64
 2   sp3_6_totalvolume   468991 non-null  float64
 3   sp6_12_pavgprice    468991 non-null  float64
 4   sp6_12_meanvolume   468991 non-null  float64
 5   sp6_12_totalvolume  468991 non-null  float64
dtypes: float64(6)
memory usage: 21.5 MB


In [68]:
drop_cols = [x for x in list(df_modelAVG.columns) if x.startswith('sp')==True]
drop_cols.remove('sp_pavgprice')
drop_cols[:5]

['sp3_pavgprice',
 'sp3_meanvolume',
 'sp3_totalvolume',
 'sp4_pavgprice',
 'sp4_meanvolume']

In [69]:
df_modelAVG = df_modelAVG.drop(columns=drop_cols, axis=1)

In [70]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   recordtype                     468991 non-null  category
 1   settlementDate                 468991 non-null  category
 2   settlementPeriod               468991 non-null  category
 3   BMU GSP Group Name             468991 non-null  category
 4   sp_pavgprice                   468991 non-null  float64 
 5   LOC Center LAT                 468991 non-null  float64 
 6   LOC Center LONG                468991 non-null  float64 
 7   Weekend                        468991 non-null  float64 
 8   Holiday                        468991 non-null  float64 
 9   Year                           468991 non-null  category
 10  Month                          468991 non-null  category
 11  Day                            468991 non-null  category
 12  Weekday_SIN     

In [71]:
df_modelAVG = pd.concat([df_modelAVG, df_aux], axis=1)

In [72]:
df_modelAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   recordtype                     468991 non-null  category
 1   settlementDate                 468991 non-null  category
 2   settlementPeriod               468991 non-null  category
 3   BMU GSP Group Name             468991 non-null  category
 4   sp_pavgprice                   468991 non-null  float64 
 5   LOC Center LAT                 468991 non-null  float64 
 6   LOC Center LONG                468991 non-null  float64 
 7   Weekend                        468991 non-null  float64 
 8   Holiday                        468991 non-null  float64 
 9   Year                           468991 non-null  category
 10  Month                          468991 non-null  category
 11  Day                            468991 non-null  category
 12  Weekday_SIN     

## 2. DEF. DATASETS "TRAIN" Y "TEST"

#### - Outliers (BID/OFFER MODEL)

In [73]:
#file_path1 = '/content/drive/MyDrive/QEnergy/Data/BIDS (outliers indexes SP).csv'
#file_path2 = '/content/drive/MyDrive/QEnergy/Data/OFFERS (outliers indexes SP).csv'

In [74]:
file_path1 = '/content/drive/MyDrive/QEnergy/Data/Outliers indexes SP & GSP (BID).csv'
file_path2 = '/content/drive/MyDrive/QEnergy/Data/Outliers indexes SP & GSP (OFFER).csv'

In [75]:
df_outBid = pd.read_csv(file_path1)
df_outOff = pd.read_csv(file_path2)

In [76]:
list_outBid = list(df_outBid.iloc[:,0])
list_outOff = list(df_outOff.iloc[:,0])

In [77]:
out_indlist = list_outBid.copy()
out_indlist.extend(list_outOff)

In [78]:
out_indlist[:5]

[369976, 369977, 416547, 416592, 453537]

In [79]:
df_model.shape

(1187159, 77)

In [80]:
df_model = df_model.drop(index=out_indlist, axis=0)

In [81]:
df_model.shape

(1186559, 77)

#### - ORDEN DEL DATASET INICIAL (BID/OFFER MODEL)

In [82]:
df_model = df_model.sort_values(by=['settlementDate', 'settlementPeriod'], ascending=True)
df_model = df_model.reset_index(drop=True)

In [83]:
df_model[df_model['settlementDate']=='2021-01-01'].loc[:50,]

,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,BID,2021-01-01,1,37.00,-18.366,E,CCGT,Eastern England,52.583834,1.733725,...,3300.832,19.125,80.5,51.8,22.893185,0.000000,0.000,27.104873,0.000000,0.000
1,BID,2021-01-01,1,35.00,-0.666,E,CCGT,South Eastern England,50.829511,-0.229161,...,3300.832,19.125,80.5,51.8,37.407064,-30.982417,-451.889,37.889961,-33.565774,-914.094
2,BID,2021-01-01,1,40.00,-11.084,T,CCGT,East Midlands,53.307421,-0.786058,...,3300.832,19.125,80.5,51.8,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
3,BID,2021-01-01,1,37.10,-174.768,T,CCGT,Southern England,50.898831,-1.437187,...,3300.832,19.125,80.5,51.8,38.280000,-92.192250,-368.769,37.630000,-89.602571,-627.218
4,BID,2021-01-01,1,37.00,-33.634,T,CCGT,Southern Wales,51.683003,-4.994865,...,3300.832,19.125,80.5,51.8,35.535761,-23.795667,-230.932,38.187343,-15.829571,-110.807
5,BID,2021-01-01,1,6.00,-49.000,T,CCGT,Yorkshire,53.735187,-0.243281,...,3300.832,19.125,80.5,51.8,22.893185,-62.276250,-747.315,27.104873,-47.949614,-1029.659
6,BID,2021-01-01,1,0.00,-299.000,T,CCGT,Yorkshire,53.602454,-0.144829,...,3300.832,19.125,80.5,51.8,22.893185,-62.276250,-747.315,27.104873,-47.949614,-1029.659
7,OFFER,2021-01-01,1,74.90,248.334,T,CCGT,North Western England,53.436959,-2.408212,...,3300.832,19.125,80.5,51.8,74.900000,95.864625,766.917,82.905750,64.704214,905.859
8,OFFER,2021-01-01,1,71.90,322.584,T,CCGT,East Midlands,53.307421,-0.786058,...,3300.832,19.125,80.5,51.8,73.322564,0.002250,0.009,93.948627,0.001286,0.009
9,OFFER,2021-01-01,1,72.00,270.000,T,CCGT,South Eastern England,51.425650,0.600555,...,3300.832,19.125,80.5,51.8,73.096574,125.464062,702.425,95.183547,13.761857,96.333


#### - ORDEN DEL DATASET INICIAL (SP PAVG. PRICE MODEL)

In [84]:
df_modelAVG = df_modelAVG.sort_values(by=['BMU GSP Group Name','settlementDate','settlementPeriod', 'recordtype'], ascending=True)
df_modelAVG = df_modelAVG.reset_index(drop=True)

In [85]:
df_modelAVG[df_modelAVG['settlementDate']=='2021-01-01'].loc[:50,]

,recordtype,settlementDate,settlementPeriod,BMU GSP Group Name,sp_pavgprice,LOC Center LAT,LOC Center LONG,Weekend,Holiday,Year,...,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price,sp3_6_pavgprice,sp3_6_meanvolume,sp3_6_totalvolume,sp6_12_pavgprice,sp6_12_meanvolume,sp6_12_totalvolume
0,BID,2021-01-01,1,East Midlands,40.000000,52.800048,-0.927751,0.0,1.0,2021,...,3300.832,19.125,80.5,51.8,36.984800,-31.193208,-205.419,38.361337,-42.150176,-1141.072
1,OFFER,2021-01-01,1,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3300.832,19.125,80.5,51.8,73.322564,0.002250,0.009,93.948627,0.001286,0.009
2,OFFER,2021-01-01,2,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3300.832,19.125,80.5,51.8,77.072564,16.968750,67.875,88.109188,0.001286,0.009
3,OFFER,2021-01-01,3,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3282.530,19.125,80.5,51.8,73.728120,51.656250,206.625,84.133499,0.001286,0.009
4,OFFER,2021-01-01,4,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3282.530,19.125,80.5,51.8,71.737500,132.302250,529.209,80.957125,0.001286,0.009
5,OFFER,2021-01-01,5,East Midlands,71.900000,52.800048,-0.927751,0.0,1.0,2021,...,3247.227,19.125,80.5,51.8,71.425000,218.927250,875.709,77.322231,9.697714,67.884
6,OFFER,2021-01-01,6,East Midlands,71.850000,52.800048,-0.927751,0.0,1.0,2021,...,3247.227,19.125,80.5,51.8,71.900000,269.042000,1076.168,74.392259,29.519143,206.634
7,BID,2021-01-01,20,East Midlands,24.123334,52.800048,-0.927751,0.0,1.0,2021,...,2779.294,19.125,80.5,51.8,29.500000,0.000000,0.000,16.442772,0.000000,0.000
8,BID,2021-01-01,21,East Midlands,24.581791,52.800048,-0.927751,0.0,1.0,2021,...,2626.488,19.125,80.5,51.8,29.500000,0.000000,0.000,15.299915,0.000000,0.000
9,OFFER,2021-01-01,21,East Midlands,35.000000,52.800048,-0.927751,0.0,1.0,2021,...,2626.488,19.125,80.5,51.8,72.712500,0.000000,0.000,74.464286,0.000000,0.000


#### - LABEL & ORDINAL ENCODING (BID/OFFER MODEL)

In [86]:
df_model_label = df_model.copy()

In [87]:
cat_vars = df_model.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod', 'BMU Type',
       'BMU Fuel Type', 'BMU GSP Group Name', 'BZONE', 'Trading Unit',
       'PC Flag', 'PC Status', 'Exempt Export Flag', 'Base TU Flag',
       'FPN Flag', 'Year', 'Month', 'Day'],
      dtype='object')

In [88]:
# Separating between categorical nominal and categorical ordinal

LabelEncoder_vars = ['recordtype', 'BMU Type', 'BMU Fuel Type', 'BMU GSP Group Name', 'BZONE', 'Trading Unit',
                     'PC Flag', 'PC Status', 'Exempt Export Flag', 'Base TU Flag', 'FPN Flag']

OrdinalEncoder_vars = ['settlementDate', 'settlementPeriod', 'Year', 'Month', 'Day']


In [89]:
for x in LabelEncoder_vars:

  le = LabelEncoder()

  new_var = x + '_label'
  df_model_label[new_var] = le.fit_transform(df_model_label[x])

In [90]:
for x in OrdinalEncoder_vars:

  order = list(df_model_label[x].unique())
  order.sort()

  aux_dict = {}
  for y in order:
    aux_dict[y] = order.index(y)

  maplist = [{'col':x, 'mapping':aux_dict}]
  oe = OrdinalEncoder(mapping=maplist)

  new_var = x + '_ordin'
  df_model_label[new_var] = oe.fit_transform(df_model_label[x])

In [91]:
df_model_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186559 entries, 0 to 1186558
Data columns (total 93 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1186559 non-null  category
 1   settlementDate                 1186559 non-null  category
 2   settlementPeriod               1186559 non-null  category
 3   acceptedprice                  1186559 non-null  float64 
 4   acceptedvolume                 1186559 non-null  float64 
 5   BMU Type                       1186559 non-null  category
 6   BMU Fuel Type                  1186559 non-null  category
 7   BMU GSP Group Name             1186559 non-null  category
 8   LOC LAT                        1186559 non-null  float64 
 9   LOC LONG                       1186559 non-null  float64 
 10  LOC Center LAT                 1186559 non-null  float64 
 11  LOC Center LONG                1186559 non-null  float64 
 12  

#### - LABEL & ORDINAL ENCODING (BID/OFFER MODEL)

In [92]:
df_modelAVG_label = df_modelAVG.copy()

In [93]:
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod',
       'BMU GSP Group Name', 'Year', 'Month', 'Day'],
      dtype='object')

In [94]:
# Separating between categorical nominal and categorical ordinal

LabelEncoder_vars = ['recordtype', 'BMU GSP Group Name']

OrdinalEncoder_vars = ['settlementDate', 'settlementPeriod', 'Year', 'Month', 'Day']


In [95]:
for x in LabelEncoder_vars:

  le = LabelEncoder()

  new_var = x + '_label'
  df_modelAVG_label[new_var] = le.fit_transform(df_modelAVG_label[x])

In [96]:
for x in OrdinalEncoder_vars:

  order = list(df_modelAVG_label[x].unique())
  order.sort()

  aux_dict = {}
  for y in order:
    aux_dict[y] = order.index(y)

  maplist = [{'col':x, 'mapping':aux_dict}]
  oe = OrdinalEncoder(mapping=maplist)

  new_var = x + '_ordin'
  df_modelAVG_label[new_var] = oe.fit_transform(df_modelAVG_label[x])

In [97]:
df_modelAVG_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468991 entries, 0 to 468990
Data columns (total 50 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   recordtype                     468991 non-null  category
 1   settlementDate                 468991 non-null  category
 2   settlementPeriod               468991 non-null  category
 3   BMU GSP Group Name             468991 non-null  category
 4   sp_pavgprice                   468991 non-null  float64 
 5   LOC Center LAT                 468991 non-null  float64 
 6   LOC Center LONG                468991 non-null  float64 
 7   Weekend                        468991 non-null  float64 
 8   Holiday                        468991 non-null  float64 
 9   Year                           468991 non-null  category
 10  Month                          468991 non-null  category
 11  Day                            468991 non-null  category
 12  Weekday_SIN     

## 2b. PREVIOUS WORKFLOW

#### - LABEL ENCODING (SP PAVG. PRICE MODEL)

In [ ]:
df_modelAVG_label = df_modelAVG.copy()

In [ ]:
cat_vars = df_modelAVG.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod',
       'BMU GSP Group Name', 'Year', 'Month', 'Day'],
      dtype='object')

In [ ]:
le = LabelEncoder()

for x in cat_vars:

  new_var = x + '_label'
  df_modelAVG_label[new_var] = le.fit_transform(df_modelAVG_label[x])


In [ ]:
df_modelAVG_label

,recordtype,settlementDate,settlementPeriod,BMU GSP Group Name,sp_pavgprice,marketIndexPrice,marketIndexVolume,LOC Center LAT,LOC Center LONG,Weekend,...,sp48_pavgprice,sp48_meanvolume,sp48_totalvolume,recordtype_label,settlementDate_label,settlementPeriod_label,BMU GSP Group Name_label,Year_label,Month_label,Day_label
0,BID,2021-01-01,1,East Midlands,40.000000,68.54,407.30,52.800048,-0.927751,0.0,...,36.221377,0.000000,0.000,0,0,0,0,0,0,0
1,OFFER,2021-01-01,1,East Midlands,71.900000,68.54,407.30,52.800048,-0.927751,0.0,...,69.149419,77.458000,154.916,1,0,0,0,0,0,0
2,OFFER,2021-01-01,2,East Midlands,71.900000,69.82,601.40,52.800048,-0.927751,0.0,...,67.989355,124.465000,248.930,1,0,1,0,0,0,0
3,OFFER,2021-01-01,3,East Midlands,71.900000,67.42,608.00,52.800048,-0.927751,0.0,...,67.750000,146.866000,146.866,1,0,2,0,0,0,0
4,OFFER,2021-01-01,4,East Midlands,71.900000,65.45,579.35,52.800048,-0.927751,0.0,...,67.865337,63.564000,127.128,1,0,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471661,BID,2023-06-30,46,Yorkshire,4.103535,85.35,2492.25,53.670497,-1.071275,0.0,...,62.792642,-18.524667,-55.574,0,909,45,13,2,5,29
471662,OFFER,2023-06-30,46,Yorkshire,169.000000,85.35,2492.25,53.670497,-1.071275,0.0,...,135.000000,2.050000,2.050,1,909,45,13,2,5,29
471663,BID,2023-06-30,47,Yorkshire,-3.207901,53.20,1380.95,53.670497,-1.071275,0.0,...,62.443450,-50.716000,-202.864,0,909,46,13,2,5,29
471664,BID,2023-06-30,48,Yorkshire,-47.158350,52.02,1497.55,53.670497,-1.071275,0.0,...,62.177127,-33.735667,-101.207,0,909,47,13,2,5,29


#### - LABEL ENCODING (BID/OFFER MODEL)

In [ ]:
df_model_label = df_model.copy()

In [ ]:
cat_vars = df_model.select_dtypes(exclude=["number"]).columns
cat_vars

Index(['recordtype', 'settlementDate', 'settlementPeriod', 'BMU Type',
       'BMU Fuel Type', 'BMU GSP Group Name', 'BZONE', 'Trading Unit',
       'PC Flag', 'PC Status', 'Exempt Export Flag', 'Base TU Flag',
       'FPN Flag', 'Year', 'Month', 'Day'],
      dtype='object')

In [ ]:
le = LabelEncoder()

for x in cat_vars:

  new_var = x + '_label'
  df_model_label[new_var] = le.fit_transform(df_model_label[x])


In [ ]:
df_model_label

,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,BZONE_label,Trading Unit_label,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,Year_label,Month_label,Day_label
0,BID,2021-01-01,1,37.00,-18.366,E,CCGT,Eastern England,52.583834,1.733725,...,3,29,1,1,0,0,1,0,0,0
1,BID,2021-01-01,1,35.00,-0.666,E,CCGT,South Eastern England,50.829511,-0.229161,...,7,29,1,1,0,0,1,0,0,0
2,OFFER,2021-01-01,1,74.90,248.334,T,CCGT,North Western England,53.436959,-2.408212,...,16,29,1,1,0,0,1,0,0,0
3,BID,2021-01-01,1,40.00,-11.084,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,0,0,0
4,OFFER,2021-01-01,1,71.90,322.584,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192752,BID,2023-06-30,48,-79.33,-81.150,T,WIND,East Midlands,53.480000,0.840000,...,1,29,1,1,0,0,1,2,5,29
1192753,BID,2023-06-30,48,-75.00,-2.261,T,WIND,Southern Scotland,55.347448,-3.880540,...,13,29,1,1,0,0,1,2,5,29
1192754,BID,2023-06-30,48,-75.00,-20.917,T,WIND,Southern Scotland,55.274700,-4.178600,...,13,29,1,1,0,0,1,2,5,29
1192755,BID,2023-06-30,48,-80.00,-1.875,T,WIND,Southern Scotland,55.245642,-4.307956,...,13,29,1,1,0,0,1,2,5,29


#### - SEPARACIÓN DATOS TRAIN Y TEST (BID/OFFER MODEL)

In [ ]:
df_model_label['settlementDate'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1192757 entries, 0 to 1192756
Series name: settlementDate
Non-Null Count    Dtype   
--------------    -----   
1192757 non-null  category
dtypes: category(1)
memory usage: 2.3 MB


In [ ]:
DataTrain_df = df_model_label[df_model_label['settlementDate']<'2023-06-30'].copy()
#DataTrain_df.drop(axis=1, columns=drop_vars, level=None, inplace=True, errors='raise')

In [ ]:
print(DataTrain_df.shape)
DataTrain_df.info()

(1191636, 92)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1191636 entries, 0 to 1191635
Data columns (total 92 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   recordtype                     1191636 non-null  category
 1   settlementDate                 1191636 non-null  category
 2   settlementPeriod               1191636 non-null  category
 3   acceptedprice                  1191636 non-null  float64 
 4   acceptedvolume                 1191636 non-null  float64 
 5   marketIndexPrice               1191636 non-null  float64 
 6   marketIndexVolume              1191636 non-null  float64 
 7   BMU Type                       1191636 non-null  category
 8   BMU Fuel Type                  1191636 non-null  category
 9   BMU GSP Group Name             1191636 non-null  category
 10  LOC LAT                        1191636 non-null  float64 
 11  LOC LONG                       1191636 non-null  

In [ ]:
DataTest_df = df_model_label[df_model_label['settlementDate']>='2023-06-30'].copy()
#DataTest_df.drop(axis=1, columns=drop_vars, level=None, inplace=True, errors='raise')

In [ ]:
print(DataTest_df.shape)
DataTest_df.info()

(1121, 92)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121 entries, 1191636 to 1192756
Data columns (total 92 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   recordtype                     1121 non-null   category
 1   settlementDate                 1121 non-null   category
 2   settlementPeriod               1121 non-null   category
 3   acceptedprice                  1121 non-null   float64 
 4   acceptedvolume                 1121 non-null   float64 
 5   marketIndexPrice               1121 non-null   float64 
 6   marketIndexVolume              1121 non-null   float64 
 7   BMU Type                       1121 non-null   category
 8   BMU Fuel Type                  1121 non-null   category
 9   BMU GSP Group Name             1121 non-null   category
 10  LOC LAT                        1121 non-null   float64 
 11  LOC LONG                       1121 non-null   float64 
 12  LOC Center LAT

#### - TRAIN & TEST (BID/OFFER MODEL)

Dividimos a su vez ambos datasets en variables explicativas y variable target:

In [ ]:
X_train = DataTrain_df.drop(axis=1, columns='acceptedprice', level=None, inplace=False, errors='raise')
y_train = DataTrain_df['acceptedprice']
print(X_train.shape)
print(y_train.shape)

(1191636, 91)
(1191636,)


In [ ]:
X_train

,recordtype,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
0,BID,2021-01-01,1,-18.366,68.54,407.30,E,CCGT,Eastern England,52.583834,...,1,1,0,0,1,0,0,0,0,0
1,BID,2021-01-01,1,-0.666,68.54,407.30,E,CCGT,South Eastern England,50.829511,...,1,1,0,0,1,0,0,0,0,0
2,OFFER,2021-01-01,1,248.334,68.54,407.30,T,CCGT,North Western England,53.436959,...,1,1,0,0,1,0,0,0,0,0
3,BID,2021-01-01,1,-11.084,68.54,407.30,T,CCGT,East Midlands,53.307421,...,1,1,0,0,1,0,0,0,0,0
4,OFFER,2021-01-01,1,322.584,68.54,407.30,T,CCGT,East Midlands,53.307421,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191631,OFFER,2023-06-29,48,120.000,74.72,770.85,T,CCGT,South Western England,51.539486,...,1,1,0,0,1,908,47,2,5,28
1191632,BID,2023-06-29,48,-84.404,74.72,770.85,T,CCGT,Yorkshire,53.602454,...,1,1,0,0,1,908,47,2,5,28
1191633,BID,2023-06-29,48,-7.009,74.72,770.85,T,WIND,Northern Scotland,58.510313,...,1,1,0,0,1,908,47,2,5,28
1191634,BID,2023-06-29,48,-2.486,74.72,770.85,T,WIND,Southern Scotland,55.347448,...,1,1,0,0,1,908,47,2,5,28


In [ ]:
y_train

0           37.00
1           35.00
2           74.90
3           40.00
4           71.90
            ...  
1191631    160.00
1191632     62.00
1191633    -71.77
1191634    -75.00
1191635    -75.00
Name: acceptedprice, Length: 1191636, dtype: float64

In [ ]:
X_test = DataTest_df.drop(axis=1, columns='acceptedprice', level=None, inplace=False, errors='raise')
#X_test.reset_index(level=None, drop=True, inplace=True)
y_test = DataTest_df['acceptedprice']
#y_test.reset_index(level=None, drop=True, inplace=True)
print(X_test.shape)
print(y_test.shape)

(1121, 91)
(1121,)


In [ ]:
X_test.head(5)

,recordtype,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
1191636,OFFER,2023-06-30,1,8.167,69.81,1072.4,2,BATTERY,East Midlands,52.800048,...,1,0,0,1,1,909,0,2,5,29
1191637,BID,2023-06-30,1,-2.950,69.81,1072.4,E,WIND,Northern Scotland,58.568706,...,2,1,1,1,1,909,0,2,5,29
1191638,BID,2023-06-30,1,-1.000,69.81,1072.4,E,WIND,Southern Scotland,55.569968,...,0,0,1,1,1,909,0,2,5,29
1191639,BID,2023-06-30,1,-4.492,69.81,1072.4,E,WIND,Northern Scotland,55.555764,...,2,1,1,1,1,909,0,2,5,29
1191640,BID,2023-06-30,1,-2.500,69.81,1072.4,E,WIND,Northern Scotland,57.363367,...,0,0,1,1,1,909,0,2,5,29


#### - TRAIN & TEST (BID/OFFER MODEL - BID)

Dividimos a su vez ambos datasets en variables explicativas y variable target:

In [ ]:
Xbid_train = DataTrain_df[DataTrain_df['recordtype']=='BID']
Xbid_train = Xbid_train.drop(axis=1, columns=['recordtype', 'recordtype_label','acceptedprice'], level=None, inplace=False, errors='raise')
ybid_train = DataTrain_df[DataTrain_df['recordtype']=='BID']['acceptedprice']
print(Xbid_train.shape)
print(ybid_train.shape)

(745115, 89)
(745115,)


In [ ]:
Xbid_train

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
0,2021-01-01,1,-18.366,68.54,407.30,E,CCGT,Eastern England,52.583834,1.733725,...,1,1,0,0,1,0,0,0,0,0
1,2021-01-01,1,-0.666,68.54,407.30,E,CCGT,South Eastern England,50.829511,-0.229161,...,1,1,0,0,1,0,0,0,0,0
3,2021-01-01,1,-11.084,68.54,407.30,T,CCGT,East Midlands,53.307421,-0.786058,...,1,1,0,0,1,0,0,0,0,0
10,2021-01-01,1,-174.768,68.54,407.30,T,CCGT,Southern England,50.898831,-1.437187,...,1,1,0,0,1,0,0,0,0,0
12,2021-01-01,1,-33.634,68.54,407.30,T,CCGT,Southern Wales,51.683003,-4.994865,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191630,2023-06-29,48,-11.086,74.72,770.85,T,WIND,Southern Scotland,55.368624,-4.042596,...,1,1,0,0,1,908,47,2,5,28
1191632,2023-06-29,48,-84.404,74.72,770.85,T,CCGT,Yorkshire,53.602454,-0.144829,...,1,1,0,0,1,908,47,2,5,28
1191633,2023-06-29,48,-7.009,74.72,770.85,T,WIND,Northern Scotland,58.510313,-4.021024,...,1,1,0,0,1,908,47,2,5,28
1191634,2023-06-29,48,-2.486,74.72,770.85,T,WIND,Southern Scotland,55.347448,-3.880540,...,1,1,0,0,1,908,47,2,5,28


In [ ]:
ybid_train

0          37.00
1          35.00
3          40.00
10         37.10
12         37.00
           ...  
1191630   -75.00
1191632    62.00
1191633   -71.77
1191634   -75.00
1191635   -75.00
Name: acceptedprice, Length: 745115, dtype: float64

In [ ]:
Xbid_test = DataTest_df[DataTest_df['recordtype']=='BID']
Xbid_test = Xbid_test.drop(axis=1, columns=['recordtype','acceptedprice'], level=None, inplace=False, errors='raise')
ybid_test = DataTest_df[DataTest_df['recordtype']=='BID']['acceptedprice']
print(Xbid_test.shape)
print(ybid_test.shape)

(875, 90)
(875,)


In [ ]:
Xbid_test.head(5)

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
1191637,2023-06-30,1,-2.950,69.81,1072.4,E,WIND,Northern Scotland,58.568706,-3.676014,...,2,1,1,1,1,909,0,2,5,29
1191638,2023-06-30,1,-1.000,69.81,1072.4,E,WIND,Southern Scotland,55.569968,-5.622702,...,0,0,1,1,1,909,0,2,5,29
1191639,2023-06-30,1,-4.492,69.81,1072.4,E,WIND,Northern Scotland,55.555764,-5.576596,...,2,1,1,1,1,909,0,2,5,29
1191640,2023-06-30,1,-2.500,69.81,1072.4,E,WIND,Northern Scotland,57.363367,-2.963840,...,0,0,1,1,1,909,0,2,5,29
1191642,2023-06-30,1,-45.000,69.81,1072.4,E,CCGT,South Eastern England,50.829511,-0.229161,...,1,1,0,0,1,909,0,2,5,29


In [ ]:
ybid_test

1191637   -71.50
1191638   -71.50
1191639   -21.53
1191640   -68.00
1191642    63.11
           ...  
1192752   -79.33
1192753   -75.00
1192754   -75.00
1192755   -80.00
1192756    25.00
Name: acceptedprice, Length: 875, dtype: float64

#### - TRAIN & TEST (BID/OFFER MODEL - OFFER)

Dividimos a su vez ambos datasets en variables explicativas y variable target:

In [ ]:
Xoffer_train = DataTrain_df[DataTrain_df['recordtype']=='OFFER']
Xoffer_train = Xoffer_train.drop(axis=1, columns=['recordtype', 'recordtype_label','acceptedprice'], level=None, inplace=False, errors='raise')
yoffer_train = DataTrain_df[DataTrain_df['recordtype']=='OFFER']['acceptedprice']
print(Xoffer_train.shape)
print(yoffer_train.shape)

(446521, 89)
(446521,)


In [ ]:
Xoffer_train

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
2,2021-01-01,1,248.334,68.54,407.30,T,CCGT,North Western England,53.436959,-2.408212,...,1,1,0,0,1,0,0,0,0,0
4,2021-01-01,1,322.584,68.54,407.30,T,CCGT,East Midlands,53.307421,-0.786058,...,1,1,0,0,1,0,0,0,0,0
5,2021-01-01,1,270.000,68.54,407.30,T,CCGT,South Eastern England,51.425650,0.600555,...,1,1,0,0,1,0,0,0,0,0
6,2021-01-01,1,230.188,68.54,407.30,T,CCGT,South Eastern England,51.443546,0.707775,...,1,1,0,0,1,0,0,0,0,0
7,2021-01-01,1,230.000,68.54,407.30,T,CCGT,South Eastern England,51.443546,0.707775,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191573,2023-06-29,47,0.171,78.80,831.45,T,CCGT,Yorkshire,53.639914,-0.234288,...,1,1,0,0,1,908,46,2,5,28
1191586,2023-06-29,47,115.000,78.80,831.45,T,COAL,East Midlands,52.865393,-1.255016,...,1,1,0,0,1,908,46,2,5,28
1191588,2023-06-29,47,120.000,78.80,831.45,T,CCGT,South Western England,51.539486,-2.670153,...,1,1,0,0,1,908,46,2,5,28
1191629,2023-06-29,48,115.000,74.72,770.85,T,COAL,East Midlands,52.865393,-1.255016,...,1,1,0,0,1,908,47,2,5,28


In [ ]:
yoffer_train

2           74.9
4           71.9
5           72.0
6           71.9
7           70.0
           ...  
1191573    169.0
1191586    165.0
1191588    160.0
1191629    165.0
1191631    160.0
Name: acceptedprice, Length: 446521, dtype: float64

In [ ]:
Xoffer_test = DataTest_df[DataTest_df['recordtype']=='OFFER']
Xoffer_test = Xoffer_test.drop(axis=1, columns=['recordtype','acceptedprice'], level=None, inplace=False, errors='raise')
yoffer_test = DataTest_df[DataTest_df['recordtype']=='OFFER']['acceptedprice']
print(Xoffer_test.shape)
print(yoffer_test.shape)

(246, 90)
(246,)


In [ ]:
Xoffer_test.head(5)

,settlementDate,settlementPeriod,acceptedvolume,marketIndexPrice,marketIndexVolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
1191636,2023-06-30,1,8.167,69.81,1072.4,2,BATTERY,East Midlands,52.800048,-0.927751,...,1,0,0,1,1,909,0,2,5,29
1191641,2023-06-30,1,0.900,69.81,1072.4,E,BATTERY,Yorkshire,53.799900,-0.410000,...,2,1,1,1,1,909,0,2,5,29
1191653,2023-06-30,1,12.000,69.81,1072.4,T,PS,Merseyside and Northern Wales,52.980956,-3.968825,...,1,1,0,0,1,909,0,2,5,29
1191654,2023-06-30,1,12.000,69.81,1072.4,T,PS,Merseyside and Northern Wales,52.980956,-3.968825,...,1,1,0,0,1,909,0,2,5,29
1191667,2023-06-30,1,115.000,69.81,1072.4,T,COAL,East Midlands,52.865393,-1.255016,...,1,1,0,0,1,909,0,2,5,29


In [ ]:
yoffer_test

1191636    119.36
1191641    119.00
1191653    159.75
1191654    159.90
1191667    165.00
            ...  
1192742    140.00
1192744    165.00
1192745    160.00
1192747    175.00
1192750    120.00
Name: acceptedprice, Length: 246, dtype: float64

## 3. XGBOOST (HYPERPARAMETERS)


**Control Overfitting**

When you observe high training accuracy, but low test accuracy, it is likely that you encountered overfitting problem. There are in general two ways that you can control overfitting in XGBoost:

- The first way is to directly control model complexity.

 - This includes max_depth, min_child_weight and gamma.

- The second way is to add randomness to make training robust to noise.

 - This includes subsample and colsample_bytree.

 - You can also reduce stepsize eta. Remember to increase num_round when you do so.

**Faster training performance**

There is a parameter called tree_method, set it to hist or gpu_hist for faster computation.


    params = {
      'colsample_bynode': 0.8,
      'learning_rate': 1,
      'max_depth': 5,
      'num_parallel_tree': 100,
      'objective': 'binary:logistic',
      'subsample': 0.8,
      'tree_method': 'gpu_hist'
    }


### Hiperparámetros

In [ ]:
        "n_estimators": trial.suggest_int(name="n_estimators", low=10, high=100, log=True),
        "min_samples_split": 1000,
        "max_depth":25,
        "n_jobs": -1,
        "random_state": 0

In [ ]:
        "min_samples_split": trial.suggest_int(name="min_samples_split", low=2, high=15),
        "max_features":  trial.suggest_int(name="max_features", low=8, high=60),
        "max_depth": trial.suggest_int(name="max_depth", low=1, high=800, log=True),
        "n_estimators": trial.suggest_int(name="n_estimators", low=1, high=2000, log=True),
        #"bootstrap": True,
        #"max_samples": trial.suggest_float(name="max_samples", low=0.5, high=1, log=True),
        "n_jobs": -1,
        "random_state": 314

In [ ]:
# ANÁLISIS HIPERPARÁMETROS CLASE MACHINE LEARNING
# INCREASE PREDICTIVE POWER
"n_estimators"
"max_features"
"mini_sample_leaf"
# INCREASE MODEL SPEED
"n_jobs"
"random_state"
"oob_score":True/False

In [ ]:
.oob_score_

In [ ]:
params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1),
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=2, scoring="r2")

In [ ]:
best_forest=grid.best_estimator_
best_forest

'''
RandomForestClassifier
RandomForestClassifier(max_depth=5, min_samples_leaf=5, n_estimators=50,
                       n_jobs=-1, random_state=42)
'''

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_train.columns,
    "Imp": best_forest.feature_importances_
})

In [ ]:
imp_df.sort_values(by="Imp", ascending=False)

## 4. XGBOOST (BID/OFFER MODEL)

    VIF
      ['DAIG_spnGeneration',
      'DANF_Demand',
      'DATF_Demand',
      'LOC Center LAT',
      'LOC LAT',
      'SystemTotalLoad_Consumption',
      'LOC Center LONG',
      'GenerationForecast_Production',
      'NatGas_price']

### Modelo OFFER (Predicción diaria)

In [ ]:
#test_dates = ['01','04','05','08','12','15','18','19','22','25','26','28','30']
test_dates = ['10','15','29']
recordtype = 'OFFER'

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x

  train = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
  train_indxs = list(train.index.values)

  test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)].copy()
  test_indxs = list(test.index.values)


  train_test[x] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['10', '15', '29'])

In [ ]:
train_test['10'].keys()

dict_keys(['train_indxs', 'test_indxs'])

In [ ]:
for x in list(train_test.keys()):
  print(len(train_test[x]['train_indxs']))
  print(len(train_test[x]['test_indxs']))

733852
487
736348
420
744275
840


In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

#unselect_vars = []

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['acceptedvolume',
 'marketIndexPrice',
 'marketIndexVolume',
 'LOC LAT',
 'LOC LONG']

In [ ]:
X_train = df_model_label.loc[train_test['10']['train_indxs'], select_vars]
y_train = df_model_label.loc[train_test['10']['train_indxs'], 'acceptedprice']
print(X_train.shape)

X_test = df_model_label.loc[train_test['10']['test_indxs'], select_vars]
y_test = df_model_label.loc[train_test['10']['test_indxs'],'acceptedprice']
print(X_test.shape)

(733852, 73)
(487, 73)


In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
train_scores = []
test_scores = []


for x in test_dates:

  start = time.time()

  test_d = '2023-06-' + x
  test_days.append(test_d)
  print('_'*35)
  print(f'Model XGBoost (Test Day {x})')

  X_train = df_model_label.loc[train_test[x]['train_indxs'], select_vars]
  y_train = df_model_label.loc[train_test[x]['train_indxs'], 'acceptedprice']

  X_test = df_model_label.loc[train_test[x]['test_indxs'], select_vars]
  y_test = df_model_label.loc[train_test[x]['test_indxs'],'acceptedprice']


  RF_bid.fit(X_train, y_train)


  size_train = y_train.shape[0]
  sumresid_train = sum(y_train - RF_bid.predict(X_train))
  stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
  r2_train = r2_score(y_train, RF_bid.predict(X_train))
  rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
  train_scores.append(rmse_train)
  print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
  print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

  size_test = y_test.shape[0]
  sumresid_test = sum(y_test - RF_bid.predict(X_test))
  stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
  r2_test = r2_score(y_test, RF_bid.predict(X_test))
  rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
  test_scores.append(rmse_test)
  print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
  print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


  model_imp = 'imp_' + x
  model_importances[model_imp] = RF_bid.feature_importances_

  end = time.time()
  print(timedelta(minutes=end-start))
  #print('\n')

model_scores = {'test_days':test_days,'train_scores':train_scores,'test_scores':test_scores}


___________________________________
Model XGBoost (Test Day 10)
Train scores: Score=0.9653625048632484 / R2=0.9653625048632484 / RMSE=19.25601400773443
Test scores: Score=0.835090627386625 / R2=0.835090627386625 / RMSE=20.80419839595804
1:33:35.667086
___________________________________
Model XGBoost (Test Day 15)
Train scores: Score=0.9624852328794156 / R2=0.9624852328794156 / RMSE=20.135265991350902
Test scores: Score=-35.3778572348729 / R2=-35.3778572348729 / RMSE=132.73083947344185
1:28:31.110921
___________________________________
Model XGBoost (Test Day 29)


KeyboardInterrupt: ignored

In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

,feature,imp_sum,imp_10,imp_15,imp_29
53,sp3_6_pavgprice,0.271756,0.297801,0.255150,0.262316
60,BMU Fuel Type_label,0.157583,0.153737,0.159112,0.159898
59,BMU Type_label,0.043761,0.012959,0.075573,0.042751
4,LOC LONG,0.039225,0.042140,0.040583,0.034953
9,Z1,0.037282,0.038867,0.037395,0.035584
...,...,...,...,...,...
32,Weekday_COS,0.000801,0.000940,0.000695,0.000767
25,Z17,0.000624,0.000112,0.000550,0.001210
18,Z10,0.000523,0.000694,0.000805,0.000070
23,Z15,0.000387,0.000000,0.001160,0.000000


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_10,imp_15,imp_29
53,sp3_6_pavgprice,0.271756,0.297801,0.255150,0.262316
60,BMU Fuel Type_label,0.157583,0.153737,0.159112,0.159898
59,BMU Type_label,0.043761,0.012959,0.075573,0.042751
4,LOC LONG,0.039225,0.042140,0.040583,0.034953
9,Z1,0.037282,0.038867,0.037395,0.035584
26,Transmission Loss Factor,0.032935,0.038721,0.014983,0.045100
2,marketIndexVolume,0.022887,0.015203,0.031299,0.022158
56,sp6_12_pavgprice,0.021532,0.019125,0.017909,0.027563
6,LOC Center LONG,0.021353,0.019814,0.025512,0.018732
66,Exempt Export Flag_label,0.021235,0.017913,0.018405,0.027386


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_10,imp_15,imp_29
13,Z5,0.001684,0.001936,0.001469,0.001646
5,LOC Center LAT,0.001571,0.002369,0.001233,0.001113
22,Z14,0.001562,0.001751,0.001449,0.001486
46,GenerationForecast_Production,0.001417,0.000915,0.001134,0.002202
17,Z9,0.001415,0.001116,0.001113,0.002016
58,sp6_12_totalvolume,0.001353,0.001617,0.001176,0.001265
67,Base TU Flag_label,0.001302,0.001145,0.002130,0.000630
15,Z7,0.001282,0.000735,0.001809,0.001303
29,Weekend,0.001281,0.000741,0.000998,0.002104
42,DATF_Demand,0.001245,0.001163,0.000744,0.001828


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo OFFER (Predicción por Mañana/Tarde)

In [ ]:
recordtype = 'OFFER'

#test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
test_dates = ['01','06','10','15','20','24','29']
test_periods = ['Mañana SP1-24','Tarde SP25-48']

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}


  test_sp = 'Mañana SP1-24'

  train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
  train_indxs = list(train_prev.index.values)
  test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<=24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}



  test_sp = 'Tarde SP25-48'

  train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
  train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<=24)].copy()
  train = pd.concat([train_prev, train_day])
  train_indxs = list(train.index.values)

  test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']>24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-06', '2023-06-10', '2023-06-15', '2023-06-20', '2023-06-24', '2023-06-29'])

In [ ]:
x = '2023-06-01'
print(train_test[x].keys())

dict_keys(['Mañana SP1-24', 'Tarde SP25-48'])


In [ ]:
x = '2023-06-01'
y = 'Mañana SP1-24'
#y = 'Tarde SP25-48'

In [ ]:
train = df_model_label.loc[train_test[x][y]['train_indxs'],:]
print(train.shape)
train

(729536, 90)


,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,BZONE_label,Trading Unit_label,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,Year_label,Month_label,Day_label
0,BID,2021-01-01,1,37.0,-18.366,E,CCGT,Eastern England,52.583834,1.733725,...,3,29,1,1,0,0,1,0,0,0
1,BID,2021-01-01,1,35.0,-0.666,E,CCGT,South Eastern England,50.829511,-0.229161,...,7,29,1,1,0,0,1,0,0,0
3,BID,2021-01-01,1,40.0,-11.084,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,0,0,0
10,BID,2021-01-01,1,37.1,-174.768,T,CCGT,Southern England,50.898831,-1.437187,...,7,29,1,1,0,0,1,0,0,0
12,BID,2021-01-01,1,37.0,-33.634,T,CCGT,Southern Wales,51.683003,-4.994865,...,4,31,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161221,BID,2023-05-31,48,26.0,-38.795,T,CCGT,Southern Wales,51.683003,-4.994865,...,4,31,1,1,0,0,1,2,4,30
1161222,BID,2023-05-31,48,30.5,-4.000,T,CCGT,South Western England,51.539486,-2.670153,...,4,36,1,1,0,0,1,2,4,30
1161224,BID,2023-05-31,48,20.5,-49.556,T,CCGT,Yorkshire,53.360271,-0.813505,...,1,38,1,1,0,0,1,2,4,30
1161225,BID,2023-05-31,48,20.5,-21.044,T,CCGT,Yorkshire,53.360271,-0.813505,...,15,38,1,1,0,0,1,2,4,30


In [ ]:
test = df_model_label.loc[train_test[x][y]['test_indxs'],:]
print(test.shape)
test

(264, 90)


,recordtype,settlementDate,settlementPeriod,acceptedprice,acceptedvolume,BMU Type,BMU Fuel Type,BMU GSP Group Name,LOC LAT,LOC LONG,...,BZONE_label,Trading Unit_label,PC Flag_label,PC Status_label,Exempt Export Flag_label,Base TU Flag_label,FPN Flag_label,Year_label,Month_label,Day_label
1161227,BID,2023-06-01,1,31.33,-57.764,E,CCGT,South Eastern England,50.829511,-0.229161,...,7,29,1,1,0,0,1,2,5,0
1161229,BID,2023-06-01,1,31.30,-99.500,T,CCGT,East Midlands,53.307421,-0.786058,...,1,29,1,1,0,0,1,2,5,0
1161231,BID,2023-06-01,1,31.30,-36.000,T,CCGT,South Eastern England,51.443546,0.707775,...,6,25,1,1,0,0,1,2,5,0
1161232,BID,2023-06-01,1,31.30,-29.000,T,CCGT,South Eastern England,51.443546,0.707775,...,6,25,1,1,0,0,1,2,5,0
1161233,BID,2023-06-01,1,24.00,-20.248,T,CCGT,Southern Wales,51.683003,-4.994865,...,4,31,1,1,0,0,1,2,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161611,BID,2023-06-01,24,75.00,-12.667,T,CCGT,Yorkshire,53.735187,-0.243281,...,15,34,1,1,0,0,1,2,5,0
1161613,BID,2023-06-01,24,33.05,-23.608,T,CCGT,Yorkshire,53.360271,-0.813505,...,1,38,1,1,0,0,1,2,5,0
1161614,BID,2023-06-01,24,33.05,-10.336,T,CCGT,Yorkshire,53.360271,-0.813505,...,15,38,1,1,0,0,1,2,5,0
1161615,BID,2023-06-01,24,51.00,-0.333,V,BATTERY,North Western England,53.780364,-2.642276,...,16,29,0,0,0,0,1,2,5,0


In [ ]:
train_test[x][y]['train_indxs'][-1]

1161226

In [ ]:
train_test[x][y]['test_indxs'][-1]

1161616

In [ ]:
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

for x in test_dates:
  for y in test_periods:

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')
    print(len(train_test[x][y]['train_indxs']))
    print(len(train_test[x][y]['test_indxs']))

_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Mañana SP1-24)
729536
264
_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Tarde SP25-48)
729800
340
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Mañana SP1-24)
732078
182
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Tarde SP25-48)
732260
133
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Mañana SP1-24)
733852
253
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Tarde SP25-48)
734105
234
_______________________________________________________
Model XGBoost (Test: Day 2023-06-15 / Mañana SP1-24)
736348
247
_______________________________________________________
Model XGBoost (Test: Day 2023-06-15 / Tarde SP25-48)
736595
173
________________________________________

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

#unselect_vars = []

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['acceptedvolume', 'LOC LAT', 'LOC LONG', 'LOC Center LONG', 'Z1']

In [ ]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-06', '2023-06-10', '2023-06-15', '2023-06-20', '2023-06-24', '2023-06-29'])

In [ ]:
train_test['2023-06-01'].keys()

dict_keys(['Mañana SP1-24', 'Tarde SP25-48'])

In [ ]:
train_test['2023-06-01']['Mañana SP1-24'].keys()

dict_keys(['train_indxs', 'test_indxs'])

In [ ]:
X_train = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], select_vars]
y_train = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], 'acceptedprice']
print(X_train.shape)

X_test = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'], select_vars]
y_test = df_model_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'],'acceptedprice']
print(X_test.shape)

(729536, 28)
(264, 28)


In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
test_sps = []
train_scores = []
test_scores = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    sumresid_train = sum(y_train - RF_bid.predict(X_train))
    stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
    r2_train = r2_score(y_train, RF_bid.predict(X_train))
    rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
    train_scores.append(rmse_train)
    print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
    print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

    size_test = y_test.shape[0]
    sumresid_test = sum(y_test - RF_bid.predict(X_test))
    stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
    r2_test = r2_score(y_test, RF_bid.predict(X_test))
    rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
    test_scores.append(rmse_test)
    print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
    print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


    if y == 'Mañana SP1-24':
      model_imp = 'imp_' + x + 'M'
    else:
      model_imp = 'imp_' + x + 'T'
    model_importances[model_imp] = RF_bid.feature_importances_

    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_days':test_days,'test_sps':test_sps,'train_scores':train_scores,'test_scores':test_scores}


_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Mañana SP1-24)
0.8869290952241193
-1.2533265130998092
0:27:21.786919
_______________________________________________________
Model XGBoost (Test: Day 2023-06-01 / Tarde SP25-48)
0.8982082635778457
-0.9201004950384222
0:26:56.863589
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Mañana SP1-24)
0.8994777606947875
-1.333947801934912
0:27:20.676212
_______________________________________________________
Model XGBoost (Test: Day 2023-06-06 / Tarde SP25-48)
0.8834751388163431
-0.07634411110709971
0:26:47.568913
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Mañana SP1-24)
0.8840903058677078
0.7222504522673059
0:26:43.569989
_______________________________________________________
Model XGBoost (Test: Day 2023-06-10 / Tarde SP25-48)
0.9076883169864725
0.8099592267000579
0:26:55.117679
________________________

In [ ]:
#pd.DataFrame(model_scores)

,test_days,test_sps,train_scores,test_scores
0,2023-06-01,Mañana SP1-24,0.953246,-0.518197
1,2023-06-01,Tarde SP25-48,0.953329,-0.818559
2,2023-06-06,Mañana SP1-24,0.954304,-7.902843
3,2023-06-06,Tarde SP25-48,0.955386,-0.334653
4,2023-06-10,Mañana SP1-24,0.955892,0.778999
5,2023-06-10,Tarde SP25-48,0.954294,0.817314
6,2023-06-15,Mañana SP1-24,0.953824,-0.985459
7,2023-06-15,Tarde SP25-48,0.954184,-21.364026
8,2023-06-20,Mañana SP1-24,0.949789,0.137927
9,2023-06-20,Tarde SP25-48,0.952392,-2.351894


In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

In [ ]:
model_imp_df.head(20)

In [ ]:
model_imp_df.tail(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo OFFER (Predicción por SP)

In [ ]:
recordtype = 'OFFER'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-03', '2023-06-05', '2023-06-07', '2023-06-09', '2023-06-11', '2023-06-13', '2023-06-15', '2023-06-17', '2023-06-19', '2023-06-21', '2023-06-23', '2023-06-25', '2023-06-27', '2023-06-29'])

In [ ]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

431700
3
431725
4
431754
8
431800
6
431817
13
431855
1
431957
19
431987
6
432023
4
432465
4
432489
4
432521
8
432588
11
432623
15
432709
10
432776
4
432803
3
432861
8
433913
2
433944
3
433977
8
434011
17
434049
4
434088
16
434148
22
434190
9
434233
7
434775
0
434777
0
434782
10
434884
12
434897
2
434904
2
434922
9
434942
1
434976
1
435281
4
435303
1
435355
10
435398
3
435412
14
435457
6
435499
6
435523
5
435569
11
436279
33
436477
32
436632
10
436725
47
436832
25
436969
29
437131
72
437319
32
437539
29
438200
4
438256
18
438318
5
438360
3
438368
8
438393
10
438492
7
438521
10
438582
0
439144
19
439222
2
439237
1
439240
0
439241
1
439266
1
439280
8
439340
17
439589
8
439927
6
439941
0
439958
7
440064
2
440070
2
440083
6
440111
1
440114
1
440161
2
440383
3
440394
1
440404
8
440438
6
440450
7
440457
2
440505
5
440524
1
440583
6
441377
9
441403
1
441429
2
441443
1
441445
0
441456
20
441560
23
441623
19
441768
14
442133
8
442153
6
442182
2
442197
1
442201
1
442242
21
442351
10
442370
11
442

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

#unselect_vars = []

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['acceptedvolume',
 'marketIndexPrice',
 'marketIndexVolume',
 'LOC LAT',
 'LOC LONG']

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}



_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=431700 / RMSE=59.9225531255883 / MeanReal: 218.07111882269655 / MeanPred: 218.0634307861328
              Resid=3321.876100718098 / AbsResidMean=21.881657234255414 / ResidMean=0.007694871671792806 / StdResidMean=0.00013880600563244189
Test scores: N=3 / RMSE=23.226254462089173 / MeanReal: 146.33333333333334 / MeanPred: 123.40152740478516
             Resid=68.79540252685547 / AbsResidMean=22.931800842285156 / ResidMean=22.931800842285156 / StdResidMean=1.2288841581553707
0:50:42.851100
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=431725 / RMSE=60.19641964964553 / MeanReal: 218.06634558053875 / MeanPred: 218.06155395507812
              Resid=2091.382598195655 / AbsResidMean=21.922565460440655 / ResidMean=0.0048442471438705755 / StdResidMean=0.000139432322554236
Test scores: N=4 / RMSE=15.744002106770367 / MeanReal: 140.7

In [ ]:
pd.DataFrame(model_scores).round(4)

,test_day,test_sp,size_train,rmse_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,431700,59.9226,218.0711,218.0711,3321.8761,21.8817,0.0077,0.0001,3,23.2263,146.3333,123.4015,68.7954,22.9318,22.9318,1.2289
1,2023-06-01,10,431725,60.1964,218.0663,218.0663,2091.3826,21.9226,0.0048,0.0001,4,15.7440,140.7500,126.2326,58.0695,14.5174,14.5174,1.5231
2,2023-06-01,16,431754,57.7499,218.0599,218.0599,1794.0452,21.8282,0.0042,0.0001,8,16.2881,117.0000,130.6972,-109.5774,14.3452,-13.6972,1.1018
3,2023-06-01,22,431800,59.1792,218.0490,218.0490,2818.2923,21.8703,0.0065,0.0001,6,7.7254,116.1667,121.4546,-31.7278,5.5743,-5.2880,0.9387
4,2023-06-01,25,431817,59.1286,218.0449,218.0449,2770.5048,21.8659,0.0064,0.0001,13,11.7598,110.8646,120.6033,-126.6024,10.7586,-9.7386,0.5071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,446327,57.7868,215.2142,215.2142,2317.1803,21.6747,0.0052,0.0001,3,55.6727,141.6667,111.2469,91.2592,34.7130,30.4197,15.5424
131,2023-06-29,31,446359,56.8181,215.2080,215.2080,3080.2509,21.4094,0.0069,0.0001,3,16.1918,134.3333,122.9905,34.0285,14.6174,11.3428,3.8516
132,2023-06-29,37,446404,57.5372,215.1996,215.1996,1897.1134,21.3930,0.0042,0.0001,9,13.1900,129.9878,123.8688,55.0709,8.5944,6.1190,1.2983
133,2023-06-29,40,446415,57.3053,215.1976,215.1976,2386.4623,21.3579,0.0053,0.0001,1,0.5681,142.5000,141.9319,0.5681,0.5681,0.5681,0.0000


In [ ]:
model_scores_df = pd.DataFrame(model_scores).round(4)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER MODEL - SP (scores).csv'
#model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_scores_df = pd.read_csv(file_path)
model_scores_df

,test_day,test_sp,size_train,rmse_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,431700,59.9226,218.0711,218.0711,3321.8761,21.8817,0.0077,0.0001,3,23.2263,146.3333,123.4015,68.7954,22.9318,22.9318,1.2289
1,2023-06-01,10,431725,60.1964,218.0663,218.0663,2091.3826,21.9226,0.0048,0.0001,4,15.7440,140.7500,126.2326,58.0695,14.5174,14.5174,1.5231
2,2023-06-01,16,431754,57.7499,218.0599,218.0599,1794.0452,21.8282,0.0042,0.0001,8,16.2881,117.0000,130.6972,-109.5774,14.3452,-13.6972,1.1018
3,2023-06-01,22,431800,59.1792,218.0490,218.0490,2818.2923,21.8703,0.0065,0.0001,6,7.7254,116.1667,121.4546,-31.7278,5.5743,-5.2880,0.9387
4,2023-06-01,25,431817,59.1286,218.0449,218.0449,2770.5048,21.8659,0.0064,0.0001,13,11.7598,110.8646,120.6033,-126.6024,10.7586,-9.7386,0.5071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,446327,57.7868,215.2142,215.2142,2317.1803,21.6747,0.0052,0.0001,3,55.6727,141.6667,111.2469,91.2592,34.7130,30.4197,15.5424
131,2023-06-29,31,446359,56.8181,215.2080,215.2080,3080.2509,21.4094,0.0069,0.0001,3,16.1918,134.3333,122.9905,34.0285,14.6174,11.3428,3.8516
132,2023-06-29,37,446404,57.5372,215.1996,215.1996,1897.1134,21.3930,0.0042,0.0001,9,13.1900,129.9878,123.8688,55.0709,8.5944,6.1190,1.2983
133,2023-06-29,40,446415,57.3053,215.1976,215.1976,2386.4623,21.3579,0.0053,0.0001,1,0.5681,142.5000,141.9319,0.5681,0.5681,0.5681,0.0000


In [ ]:
model_scores_df.sort_values(by=['rmse_test'], ascending=False).head(20)

,test_day,test_sp,size_train,rmse_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
110,2023-06-25,16,443163,55.8793,215.7640,215.7640,2208.9662,21.4451,0.0050,0.0001,47,139.7129,126.5366,144.0970,-825.3397,29.7734,-17.5604,2.9490
130,2023-06-29,25,446327,57.7868,215.2142,215.2142,2317.1803,21.6747,0.0052,0.0001,3,55.6727,141.6667,111.2469,91.2592,34.7130,30.4197,15.5424
8,2023-06-01,48,432023,59.1062,217.9946,217.9946,3704.0285,21.7080,0.0086,0.0001,4,51.4108,160.7500,114.0113,186.9549,46.7387,46.7387,5.3535
125,2023-06-27,48,445562,57.7170,215.3420,215.3420,3456.4029,21.4000,0.0078,0.0001,8,39.1982,157.2188,134.5821,181.0933,35.5900,22.6367,4.0002
9,2023-06-03,4,432465,57.3213,217.8961,217.8961,1304.1178,21.7938,0.0030,0.0001,4,38.6318,141.2250,117.8871,93.3515,31.3773,23.3379,7.6964
118,2023-06-27,10,445235,55.9495,215.3990,215.3990,1827.7450,21.3449,0.0041,0.0001,10,37.3736,160.2200,136.1895,240.3046,30.0085,24.0305,2.8624
58,2023-06-13,25,438368,54.5721,216.6224,216.6224,2062.9394,21.4186,0.0047,0.0001,8,33.6824,132.6812,102.2239,243.6585,30.4573,30.4573,1.7978
17,2023-06-03,48,432861,57.3846,217.7996,217.7996,2361.4487,21.6782,0.0055,0.0001,8,33.0539,132.7438,116.3100,131.4701,29.4775,16.4338,3.5849
99,2023-06-23,4,442133,56.3346,215.9616,215.9616,2238.0340,21.5691,0.0051,0.0001,8,31.1285,137.7500,120.9942,134.0463,18.5151,16.7558,3.2793
134,2023-06-29,48,446519,57.2872,215.1800,215.1800,2696.2935,21.4712,0.0060,0.0001,2,30.2588,162.5000,132.2721,60.4558,30.2279,30.2279,0.6840


In [ ]:
model_scores_df['rmse_test'].mean()

15.824276744186045

In [ ]:
model_scores_df['meanreal_test'].mean()

128.88886356589148

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER MODEL - SP (feat.importances).csv'
#model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.read_csv(file_path)
model_imp_df

In [ ]:
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
3,LOC LAT,0.108194,0.111077,0.108976,0.094293,0.111124,0.115803,0.111423,0.116925,0.116341,0.100666,0.104341,0.102460,0.105127,0.111600,0.112347,0.100400
51,Coal_price,0.102679,0.098233,0.108656,0.089578,0.103072,0.106885,0.088844,0.120940,0.105630,0.090762,0.104564,0.105679,0.110572,0.100796,0.106121,0.099858
53,sp3_6_pavgprice,0.088702,0.089076,0.097358,0.084615,0.094093,0.101243,0.088123,0.091007,0.088065,0.081679,0.084478,0.076823,0.085587,0.088130,0.089743,0.090515
47,Solar_Generation,0.071809,0.073410,0.071145,0.087005,0.080972,0.074860,0.076206,0.049485,0.057248,0.064703,0.078201,0.057294,0.087257,0.075230,0.070591,0.073521
11,Z3,0.044123,0.035142,0.040654,0.039348,0.034761,0.036031,0.077446,0.027814,0.023997,0.057073,0.052951,0.037549,0.048991,0.039000,0.043393,0.067699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,Z8,0.001079,0.001092,0.002586,0.000047,0.001302,0.000745,0.000901,0.001491,0.000857,0.000496,0.000181,0.000052,0.001858,0.001532,0.000971,0.002077
10,Z2,0.000696,0.000601,0.000458,0.000000,0.000000,0.000820,0.000799,0.001411,0.000806,0.000366,0.000000,0.000933,0.000634,0.000464,0.000725,0.002429
25,Z17,0.000474,0.000000,0.000000,0.000000,0.000456,0.000000,0.000274,0.003583,0.000242,0.000007,0.000000,0.000437,0.000000,0.001237,0.000000,0.000871
30,Holiday,0.000279,0.000214,0.000065,0.000374,0.000446,0.000366,0.000157,0.000000,0.000479,0.000000,0.000005,0.000000,0.000382,0.000275,0.000605,0.000821


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
3,LOC LAT,0.108194,0.111077,0.108976,0.094293,0.111124,0.115803,0.111423,0.116925,0.116341,0.100666,0.104341,0.102460,0.105127,0.111600,0.112347,0.100400
51,Coal_price,0.102679,0.098233,0.108656,0.089578,0.103072,0.106885,0.088844,0.120940,0.105630,0.090762,0.104564,0.105679,0.110572,0.100796,0.106121,0.099858
53,sp3_6_pavgprice,0.088702,0.089076,0.097358,0.084615,0.094093,0.101243,0.088123,0.091007,0.088065,0.081679,0.084478,0.076823,0.085587,0.088130,0.089743,0.090515
47,Solar_Generation,0.071809,0.073410,0.071145,0.087005,0.080972,0.074860,0.076206,0.049485,0.057248,0.064703,0.078201,0.057294,0.087257,0.075230,0.070591,0.073521
11,Z3,0.044123,0.035142,0.040654,0.039348,0.034761,0.036031,0.077446,0.027814,0.023997,0.057073,0.052951,0.037549,0.048991,0.039000,0.043393,0.067699
7,BZONE GENERATION,0.043412,0.058484,0.046833,0.031316,0.034714,0.038237,0.044231,0.038612,0.047155,0.059281,0.033767,0.048747,0.030251,0.042197,0.047564,0.049789
38,settPeriod_COS,0.040816,0.038042,0.037483,0.040961,0.049623,0.046341,0.037373,0.046553,0.040670,0.040638,0.043503,0.030880,0.045663,0.038792,0.040858,0.034858
0,acceptedvolume,0.033279,0.034458,0.035994,0.037297,0.038867,0.038614,0.029955,0.031972,0.031907,0.029618,0.031531,0.029571,0.035665,0.032446,0.032821,0.028472
45,DAImbalance,0.027612,0.018776,0.018238,0.012322,0.018913,0.018124,0.026500,0.030424,0.033593,0.029831,0.032234,0.031019,0.037243,0.035505,0.034184,0.037269
64,PC Flag_label,0.023576,0.023410,0.025848,0.025570,0.030937,0.025146,0.024737,0.028333,0.025377,0.019808,0.020129,0.023418,0.019889,0.021217,0.020238,0.019583


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
55,sp3_6_totalvolume,0.003544,0.003714,0.004349,0.003702,0.005554,0.005622,0.004103,0.003916,0.003276,0.003157,0.002536,0.003415,0.003516,0.002054,0.002129,0.002120
41,DAIG_spnGeneration,0.003542,0.003199,0.003401,0.002346,0.003265,0.003313,0.003734,0.003093,0.004433,0.004448,0.004535,0.002748,0.003293,0.005166,0.003217,0.002938
72,Day_ordin,0.003474,0.001155,0.006912,0.001813,0.003464,0.001219,0.003340,0.003011,0.002771,0.002657,0.008261,0.002307,0.002527,0.002973,0.008072,0.001629
17,Z9,0.003386,0.001842,0.002346,0.001179,0.001971,0.001475,0.001663,0.000448,0.033623,0.001296,0.000674,0.000000,0.000853,0.000111,0.001847,0.001456
9,Z1,0.003346,0.003908,0.004416,0.002977,0.003145,0.002686,0.004378,0.003306,0.002842,0.002491,0.004010,0.002312,0.002403,0.003562,0.004064,0.003695
43,SystemTotalLoad_Consumption,0.003331,0.003635,0.003496,0.002580,0.003744,0.004355,0.002414,0.003151,0.003223,0.002457,0.003602,0.002964,0.003358,0.004140,0.004058,0.002793
59,BMU Type_label,0.003293,0.002574,0.004948,0.002571,0.002243,0.001294,0.002794,0.004171,0.004156,0.002794,0.002669,0.001357,0.006667,0.002514,0.006277,0.002369
23,Z15,0.003103,0.000502,0.000000,0.005237,0.003047,0.001556,0.000167,0.000000,0.000000,0.000000,0.003647,0.001266,0.000922,0.001411,0.001103,0.027688
29,Weekend,0.003060,0.004317,0.000350,0.000380,0.008488,0.000666,0.003319,0.000313,0.000122,0.000000,0.000791,0.026090,0.000476,0.000192,0.000016,0.000380
39,DANF_Demand,0.002763,0.003796,0.004226,0.004646,0.001834,0.002429,0.001827,0.003030,0.002361,0.002166,0.001867,0.001903,0.002787,0.002182,0.003964,0.002423


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

## 5. XGBOOST (SP PAVG. PRICE MODEL)

    VIF
      ['DAIG_spnGeneration',
      'DANF_Demand',
      'DATF_Demand',
      'SystemTotalLoad_Consumption',
      'GenerationForecast_Production',
      'NatGas_price']

### Modelo OFFER (Predicción diaria)

In [ ]:
df_modelAVG_label.head()

,recordtype,settlementDate,settlementPeriod,BMU GSP Group Name,sp_pavgprice,marketIndexPrice,marketIndexVolume,LOC Center LAT,LOC Center LONG,Weekend,...,sp36_48_pavgprice,sp36_48_meanvolume,sp36_48_totalvolume,recordtype_label,BMU GSP Group Name_label,settlementDate_ordin,settlementPeriod_ordin,Year_ordin,Month_ordin,Day_ordin
0,BID,2021-01-01,1,East Midlands,40.0,68.54,407.30,52.800048,-0.927751,0.0,...,33.812282,-0.040333,-0.040333,0,0,0,0,0,0,0
1,OFFER,2021-01-01,1,East Midlands,71.9,68.54,407.30,52.800048,-0.927751,0.0,...,68.501114,104.789792,158.941333,1,0,0,0,0,0,0
2,OFFER,2021-01-01,2,East Midlands,71.9,69.82,601.40,52.800048,-0.927751,0.0,...,68.531072,109.552083,168.465917,1,0,0,1,0,0,0
3,OFFER,2021-01-01,3,East Midlands,71.9,67.42,608.00,52.800048,-0.927751,0.0,...,68.658646,105.195042,171.990667,1,0,0,2,0,0,0
4,OFFER,2021-01-01,4,East Midlands,71.9,65.45,579.35,52.800048,-0.927751,0.0,...,68.944868,109.481375,170.980000,1,0,0,3,0,0,0


In [ ]:
#test_dates = ['01','04','05','08','12','15','18','19','22','25','26','28','30']
test_dates = ['10','15','29']

recordtype = 'OFFER'

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x

  train = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
  train_indxs = list(train.index.values)

  test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)].copy()
  test_indxs = list(test.index.values)


  train_test[x] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['10', '15', '29'])

In [ ]:
for x in list(train_test.keys()):
  print(len(train_test[x]['train_indxs']))
  print(len(train_test[x]['test_indxs']))

233338
188
234607
247
238394
360


In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',

                 'Month_label', 'Month_SIN', 'Month_COS', 'Day_label', 'Day_SIN', 'settlementPeriod_label', 'Weekday_SIN', 'Weekday_COS',
                 'Z2',

                 'Day_COS', 'settPeriod_COS'
                 ]
'''

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',\n                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',\n                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',\n\n                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',\n                 'LOC Center LAT',\n                 'Holiday',\n                 'Trading Unit_label', 'Base TU Flag_label',\n                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',\n\n                 'Z3', 'BZONE GENERATION',\n                 'settlementDate_label',\n                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',\n                 'Coal_price', 'BrentOil_price', 'NatGas_price',\n                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation',\n\n            

In [ ]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

['marketIndexPrice',
 'marketIndexVolume',
 'LOC Center LAT',
 'LOC Center LONG',
 'Weekend']

In [ ]:
X_train = df_modelAVG_label.loc[train_test['10']['train_indxs'], select_vars]
y_train = df_modelAVG_label.loc[train_test['10']['train_indxs'], 'sp_pavgprice']
print(X_train.shape)

X_test = df_modelAVG_label.loc[train_test['10']['test_indxs'], select_vars]
y_test = df_modelAVG_label.loc[train_test['10']['test_indxs'],'sp_pavgprice']
print(X_test.shape)

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
train_scores = []
test_scores = []


for x in test_dates:

  start = time.time()

  test_d = '2023-06-' + x
  test_days.append(test_d)
  print('_'*35)
  print(f'Model XGBoost (Test Day {x})')

  X_train = df_modelAVG_label.loc[train_test[x]['train_indxs'], select_vars]
  y_train = df_modelAVG_label.loc[train_test[x]['train_indxs'], 'sp_pavgprice']

  X_test = df_modelAVG_label.loc[train_test[x]['test_indxs'], select_vars]
  y_test = df_modelAVG_label.loc[train_test[x]['test_indxs'],'sp_pavgprice']


  RF_bid.fit(X_train, y_train)


  size_train = y_train.shape[0]
  sumresid_train = sum(y_train - RF_bid.predict(X_train))
  stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
  r2_train = r2_score(y_train, RF_bid.predict(X_train))
  rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
  train_scores.append(rmse_train)
  print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
  print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

  size_test = y_test.shape[0]
  sumresid_test = sum(y_test - RF_bid.predict(X_test))
  stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
  r2_test = r2_score(y_test, RF_bid.predict(X_test))
  rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
  test_scores.append(rmse_test)
  print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
  print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


  model_imp = 'imp_' + x
  #model_importances[model_imp] = RF_bid.feature_importances_

  end = time.time()
  print(timedelta(minutes=end-start))
  #print('\n')

model_scores = {'test_days':test_days,'train_scores':train_scores,'test_scores':test_scores}


In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

,feature,imp_sum
0,marketIndexPrice,NaN
1,marketIndexVolume,NaN
2,LOC Center LAT,NaN
3,Holiday,NaN
4,Weekday_SIN,NaN
5,Weekday_COS,NaN
6,Month_SIN,NaN
7,Month_COS,NaN
8,Day_SIN,NaN
9,Day_COS,NaN


In [ ]:
model_imp_df.head(20)

,feature,imp_sum
0,marketIndexPrice,NaN
1,marketIndexVolume,NaN
2,LOC Center LAT,NaN
3,Holiday,NaN
4,Weekday_SIN,NaN
5,Weekday_COS,NaN
6,Month_SIN,NaN
7,Month_COS,NaN
8,Day_SIN,NaN
9,Day_COS,NaN


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum
19,NatGas_price,NaN
20,Coal_price,NaN
21,BrentOil_price,NaN
22,sp3_6_meanvolume,NaN
23,sp3_6_totalvolume,NaN
24,sp6_12_meanvolume,NaN
25,sp6_12_totalvolume,NaN
26,sp12_24_pavgprice,NaN
27,sp12_24_meanvolume,NaN
28,sp12_24_totalvolume,NaN


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo OFFER (Predicción por Mañana/Tarde)

In [ ]:
recordtype = 'OFFER'

#test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
test_dates = ['01','06','10','15','20','24','29']
test_periods = ['Mañana SP1-24','Tarde SP25-48']

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}


  test_sp = 'Mañana SP1-24'

  train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
  train_indxs = list(train_prev.index.values)
  test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<=24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}



  test_sp = 'Tarde SP25-48'

  train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
  train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==df_modelAVG_label)&(df_modelAVG_label['settlementPeriod']<=24)].copy()
  train = pd.concat([train_prev, train_day])
  train_indxs = list(train.index.values)

  test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']>24)].copy()
  test_indxs = list(test.index.values)

  train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

In [ ]:
x = '2023-06-01'
print(train_test[x].keys())

In [ ]:
x = '2023-06-01'
y = 'Mañana SP1-24'
#y = 'Tarde SP25-48'

In [ ]:
train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'],:]
print(train.shape)
train

In [ ]:
df_modelAVG_label.loc[train_test[x][y]['test_indxs'],:]
print(test.shape)
test

In [ ]:
train_test[x][y]['train_indxs'][-1]

In [ ]:
train_test[x][y]['test_indxs'][-1]

In [ ]:
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

for x in test_dates:
  for y in test_periods:

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')
    print(len(train_test[x][y]['train_indxs']))
    print(len(train_test[x][y]['test_indxs']))

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['Z5', 'Z7', 'Z9', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z17',
                 'Year_label', 'FPN Flag_label', 'PC Status_label', 'BMU GSP Group Name_label',
                 'BZONE DEMAND', 'DATF_Demand', 'Solar_Generation', 'SystemTotalLoad',

                 'Z4', 'Z6', 'Z8', 'Z16', 'BZONE_label',
                 'LOC Center LAT',
                 'Holiday',
                 'Trading Unit_label', 'Base TU Flag_label',
                 'SystemTotalLoad_Consumption', 'GenerationForecast_Production', 'WindOffshore_Generation',

                 'Z3', 'BZONE GENERATION',
                 'settlementDate_label',
                 'sp6_12_meanvolume', 'sp6_12_totalvolume', 'sp3_6_meanvolume', 'sp3_6_totalvolume',
                 'Coal_price', 'BrentOil_price', 'NatGas_price',
                 'PC Flag_label', 'DAIG_spnGeneration', 'Generation Capacity', 'DAMargin', 'WindOnshore_Generation']
'''

In [ ]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

73

In [ ]:
select_vars[:5]

In [ ]:
train_test.keys()

In [ ]:
train_test['2023-06-01'].keys()

In [ ]:
train_test['2023-06-01']['Mañana SP1-24'].keys()

In [ ]:
X_train = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], select_vars]
y_train = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['train_indxs'], 'sp_pavgprice']
print(X_train.shape)

X_test = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'], select_vars]
y_test = df_modelAVG_label.loc[train_test['2023-06-01']['Mañana SP1-24']['test_indxs'],'sp_pavgprice']
print(X_test.shape)

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}
model_importances = {'feature':select_vars}

test_days = []
test_sps = []
train_scores = []
test_scores = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*55)
    print(f'Model XGBoost (Test: Day {x} / {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    sumresid_train = sum(y_train - RF_bid.predict(X_train))
    stdresid_train = np.std(y_train - RF_bid.predict(X_train)) / size_train
    r2_train = r2_score(y_train, RF_bid.predict(X_train))
    rmse_train = sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))
    train_scores.append(rmse_train)
    print(f'Train scores: N={size_train} / SumResid={sumresid_train} / SumResid/N={sumresid_train/size_train} / StdResid/N={stdresid_train} / RMSE={rmse_train}')
    print(f'              Avg. Real: {y_train.mean()} - Avg. Pred: {RF_bid.predict(X_train).mean()}')

    size_test = y_test.shape[0]
    sumresid_test = sum(y_test - RF_bid.predict(X_test))
    stdresid_test = np.std(y_test - RF_bid.predict(X_test)) / size_test
    r2_test = r2_score(y_test, RF_bid.predict(X_test))
    rmse_test = sqrt(mean_squared_error(y_test, RF_bid.predict(X_test)))
    test_scores.append(rmse_test)
    print(f'Test scores: N={size_test} / SumResid={sumresid_test} / SumResid/N={sumresid_test/size_test} / StdResid/N={stdresid_test} / RMSE={rmse_test}')
    print(f'             Avg. Real: {y_test.mean()} - Avg. Pred: {RF_bid.predict(X_test).mean()}')


    if y == 'Mañana SP1-24':
      model_imp = 'imp_' + x + 'M'
    else:
      model_imp = 'imp_' + x + 'T'
    model_importances[model_imp] = RF_bid.feature_importances_

    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_days':test_days,'test_sps':test_sps,'train_scores':train_scores,'test_scores':test_scores}


In [ ]:
#pd.DataFrame(model_scores)

In [ ]:
pd.DataFrame(model_scores)

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (scores).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/Model Day (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

In [ ]:
model_imp_df.head(20)

In [ ]:
model_imp_df.tail(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### Modelo OFFER (Predicción por SP)

In [ ]:
recordtype = 'OFFER'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
train_test.keys()

dict_keys(['2023-06-01', '2023-06-03', '2023-06-05', '2023-06-07', '2023-06-09', '2023-06-11', '2023-06-13', '2023-06-15', '2023-06-17', '2023-06-19', '2023-06-21', '2023-06-23', '2023-06-25', '2023-06-27', '2023-06-29'])

In [ ]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

224966
3
224989
3
225013
6
225048
5
225062
7
225086
1
225139
10
225159
5
225194
4
225452
3
225471
3
225497
6
225544
7
225567
9
225617
6
225666
4
225687
3
225728
7
226162
2
226187
3
226214
5
226236
9
226259
4
226281
5
226314
11
226336
7
226370
6
226648
0
226650
0
226655
5
226703
5
226709
2
226716
2
226730
6
226747
1
226778
1
226960
4
226979
1
227008
7
227036
2
227048
10
227081
5
227103
4
227120
3
227149
7
227541
13
227617
12
227681
8
227724
13
227756
8
227813
9
227864
13
227903
13
227983
10
228325
4
228358
11
228396
4
228424
3
228431
5
228451
6
228506
6
228528
8
228574
0
228828
11
228873
2
228886
1
228889
0
228890
1
228907
1
228917
4
228942
8
229029
5
229228
5
229241
0
229253
5
229296
1
229301
2
229313
6
229336
1
229339
1
229366
1
229528
3
229539
1
229548
6
229568
4
229576
4
229580
2
229608
4
229620
1
229663
4
229983
5
230002
1
230020
1
230030
1
230032
0
230042
8
230081
10
230108
10
230176
9
230410
4
230426
4
230447
2
230460
1
230464
1
230486
9
230541
8
230556
6
230584
3
230876
8
230935

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

'''
'', '', '', '', '', '', '', '', '', '',

unselect_vars = ['recordtype_label', 'Year_ordin', 'settPeriod_COS','Day_SIN',
                 'Holiday', 'Month_ordin', 'Day_ordin',
                 'sp6_12_totalvolume', 'marketIndexPrice', 'marketIndexVolume',
                 'NatGas_price', 'SystemTotalLoad_Consumption', 'sp3_6_totalvolume', 'BrentOil_price', 'WindOffshore_Generation',
                 'DAMargin', 'sp6_12_meanvolume', 'GenerationForecast_Production', 'settlementDate_ordin', 'BMU GSP Group Name_label',
                 'DATF_Demand', 'settlementPeriod_ordin', 'DAImbalance', 'Weekday_COS', 'Solar_Generation', 'Weekday_SIN', 'Day_COS'
                 ]


'''

"\n'', '', '', '', '', '', '', '', '', '',\n\nunselect_vars = ['recordtype_label', 'Year_ordin', 'settPeriod_COS','Day_SIN',\n                 'Holiday', 'Month_ordin', 'Day_ordin',\n                 'sp6_12_totalvolume', 'marketIndexPrice', 'marketIndexVolume',\n                 'NatGas_price', 'SystemTotalLoad_Consumption', 'sp3_6_totalvolume', 'BrentOil_price', 'WindOffshore_Generation',\n                 'DAMargin', 'sp6_12_meanvolume', 'GenerationForecast_Production', 'settlementDate_ordin', 'BMU GSP Group Name_label',\n                 'DATF_Demand', 'settlementPeriod_ordin', 'DAImbalance', 'Weekday_COS', 'Solar_Generation', 'Weekday_SIN', 'Day_COS'\n                 ]\n\n\n"

In [ ]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

In [ ]:
len(select_vars)

39

In [ ]:
select_vars[:5]

['marketIndexPrice',
 'marketIndexVolume',
 'LOC Center LAT',
 'LOC Center LONG',
 'Weekend']

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1)  # n_estimators=100, max_depth=6, eta=0.3

In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []



for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}



_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=224966 / RMSE=70.23954075153695 / MeanReal: 215.8531293552703 / MeanPred: 215.8314971923828
              Resid=4873.1837218107485 / AbsResidMean=25.86779963503927 / ResidMean=0.021661867668035524 / StdResidMean=0.0003122229021775642
Test scores: N=3 / RMSE=21.494987613520596 / MeanReal: 146.33333333333334 / MeanPred: 125.47026824951172
             Resid=62.589202880859375 / AbsResidMean=20.863067626953125 / ResidMean=20.863067626953125 / StdResidMean=1.7245579690841346
0:24:05.982971
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=224989 / RMSE=68.2848179661035 / MeanReal: 215.8451588117084 / MeanPred: 215.84152221679688
              Resid=823.1813369440927 / AbsResidMean=25.525279470400665 / ResidMean=0.003658762592578148 / StdResidMean=0.00030350291733410785
Test scores: N=3 / RMSE=8.359993188218247 / MeanReal: 146.322

In [ ]:
pd.DataFrame(model_scores).round(4)

,test_day,test_sp,size_train,rmse_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,224966,70.2395,215.8531,215.8531,4873.1837,25.8678,0.0217,0.0003,3,21.4950,146.3333,125.4703,62.5892,20.8631,20.8631,1.7246
1,2023-06-01,10,224989,68.2848,215.8452,215.8452,823.1813,25.5253,0.0037,0.0003,3,8.3600,146.3230,138.1699,24.4594,8.1531,8.1531,0.6161
2,2023-06-01,16,225013,73.8666,215.8355,215.8355,2334.3761,26.0721,0.0104,0.0003,6,19.3405,113.9314,125.7729,-71.0492,13.6420,-11.8415,2.5486
3,2023-06-01,22,225048,70.4624,215.8196,215.8196,2731.7115,25.8599,0.0121,0.0003,5,5.7123,114.4000,118.6047,-21.0236,4.2047,-4.2047,0.7733
4,2023-06-01,25,225062,68.9017,215.8132,215.8132,3940.8921,25.8022,0.0175,0.0003,7,11.3916,110.7397,117.1846,-45.1147,9.5080,-6.4450,1.3419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,232204,72.4960,213.1704,213.1704,2537.5526,25.5718,0.0109,0.0003,2,8.4383,112.5036,106.6806,11.6461,6.1071,5.8230,3.0536
131,2023-06-29,31,232227,72.8349,213.1618,213.1618,1832.5370,25.6068,0.0079,0.0003,3,16.0686,134.3333,120.4279,41.7163,13.9054,13.9054,2.6841
132,2023-06-29,37,232258,72.8303,213.1513,213.1513,1833.1444,25.6047,0.0079,0.0003,7,13.1020,132.0964,128.7818,23.2018,9.9355,3.3145,1.8108
133,2023-06-29,40,232267,73.8820,213.1482,213.1482,2529.4681,25.6403,0.0109,0.0003,1,13.4504,142.5000,129.0496,13.4504,13.4504,13.4504,0.0000


In [ ]:
model_scores_df = pd.DataFrame(model_scores).round(4)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER SP PAVG. PRICE MODEL - SP (scores).csv'
#model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_scores_df = pd.read_csv(file_path)
model_scores_df

,test_day,test_sp,size_train,rmse_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,224966,70.2395,215.8531,215.8531,4873.1837,25.8678,0.0217,0.0003,3,21.4950,146.3333,125.4703,62.5892,20.8631,20.8631,1.7246
1,2023-06-01,10,224989,68.2848,215.8452,215.8452,823.1813,25.5253,0.0037,0.0003,3,8.3600,146.3230,138.1699,24.4594,8.1531,8.1531,0.6161
2,2023-06-01,16,225013,73.8666,215.8355,215.8355,2334.3761,26.0721,0.0104,0.0003,6,19.3405,113.9314,125.7729,-71.0492,13.6420,-11.8415,2.5486
3,2023-06-01,22,225048,70.4624,215.8196,215.8196,2731.7115,25.8599,0.0121,0.0003,5,5.7123,114.4000,118.6047,-21.0236,4.2047,-4.2047,0.7733
4,2023-06-01,25,225062,68.9017,215.8132,215.8132,3940.8921,25.8022,0.0175,0.0003,7,11.3916,110.7397,117.1846,-45.1147,9.5080,-6.4450,1.3419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,232204,72.4960,213.1704,213.1704,2537.5526,25.5718,0.0109,0.0003,2,8.4383,112.5036,106.6806,11.6461,6.1071,5.8230,3.0536
131,2023-06-29,31,232227,72.8349,213.1618,213.1618,1832.5370,25.6068,0.0079,0.0003,3,16.0686,134.3333,120.4279,41.7163,13.9054,13.9054,2.6841
132,2023-06-29,37,232258,72.8303,213.1513,213.1513,1833.1444,25.6047,0.0079,0.0003,7,13.1020,132.0964,128.7818,23.2018,9.9355,3.3145,1.8108
133,2023-06-29,40,232267,73.8820,213.1482,213.1482,2529.4681,25.6403,0.0109,0.0003,1,13.4504,142.5000,129.0496,13.4504,13.4504,13.4504,0.0000


In [ ]:
model_scores_df['rmse_test'].mean()

18.268916279069764

In [ ]:
model_scores_df['meanreal_test'].mean()

128.4466046511628

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER SP PAVG. PRICE MODEL - SP (feat.importances).csv'
model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
28,sp3_6_pavgprice,0.178617,0.173834,0.158554,0.176000,0.180879,0.188070,0.168369,0.176537,0.193320,0.192543,0.183530,0.174947,0.181305,0.178800,0.170122,0.182437
13,settPeriod_COS,0.148481,0.145038,0.147587,0.142489,0.137635,0.139401,0.141091,0.153474,0.152620,0.134195,0.159938,0.164752,0.150259,0.156033,0.142104,0.160600
38,Day_ordin,0.049601,0.056107,0.049506,0.045843,0.046397,0.044007,0.065534,0.061589,0.044522,0.041498,0.049684,0.063323,0.045354,0.039390,0.044902,0.046360
2,LOC Center LAT,0.039719,0.042809,0.038817,0.044652,0.039112,0.057359,0.041242,0.035594,0.038561,0.038821,0.032294,0.041837,0.038176,0.043040,0.032485,0.030996
19,DAMargin,0.037530,0.043389,0.037396,0.033627,0.034963,0.037022,0.036253,0.038670,0.035188,0.041171,0.031368,0.044815,0.039565,0.035220,0.037403,0.036902
30,sp3_6_totalvolume,0.031999,0.031549,0.028950,0.029550,0.033763,0.034931,0.030068,0.028634,0.031008,0.035520,0.030145,0.029441,0.029941,0.038050,0.035540,0.032903
20,DAImbalance,0.031774,0.035704,0.029563,0.020872,0.022836,0.036661,0.028067,0.038827,0.037190,0.041194,0.047300,0.045642,0.035043,0.020590,0.018153,0.018961
4,Weekend,0.030005,0.054143,0.024176,0.034679,0.023632,0.022105,0.051505,0.026343,0.026609,0.022041,0.021086,0.026744,0.028096,0.027275,0.025520,0.036115
29,sp3_6_meanvolume,0.029234,0.027853,0.027951,0.028922,0.027421,0.033172,0.027788,0.029979,0.035164,0.033927,0.026520,0.025272,0.028852,0.032721,0.027418,0.025546
0,marketIndexPrice,0.024364,0.027958,0.022388,0.020787,0.021378,0.019680,0.025107,0.027763,0.024891,0.026067,0.023896,0.022051,0.022342,0.028860,0.027666,0.024629


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
28,sp3_6_pavgprice,0.178617,0.173834,0.158554,0.176000,0.180879,0.188070,0.168369,0.176537,0.193320,0.192543,0.183530,0.174947,0.181305,0.178800,0.170122,0.182437
13,settPeriod_COS,0.148481,0.145038,0.147587,0.142489,0.137635,0.139401,0.141091,0.153474,0.152620,0.134195,0.159938,0.164752,0.150259,0.156033,0.142104,0.160600
38,Day_ordin,0.049601,0.056107,0.049506,0.045843,0.046397,0.044007,0.065534,0.061589,0.044522,0.041498,0.049684,0.063323,0.045354,0.039390,0.044902,0.046360
2,LOC Center LAT,0.039719,0.042809,0.038817,0.044652,0.039112,0.057359,0.041242,0.035594,0.038561,0.038821,0.032294,0.041837,0.038176,0.043040,0.032485,0.030996
19,DAMargin,0.037530,0.043389,0.037396,0.033627,0.034963,0.037022,0.036253,0.038670,0.035188,0.041171,0.031368,0.044815,0.039565,0.035220,0.037403,0.036902
30,sp3_6_totalvolume,0.031999,0.031549,0.028950,0.029550,0.033763,0.034931,0.030068,0.028634,0.031008,0.035520,0.030145,0.029441,0.029941,0.038050,0.035540,0.032903
20,DAImbalance,0.031774,0.035704,0.029563,0.020872,0.022836,0.036661,0.028067,0.038827,0.037190,0.041194,0.047300,0.045642,0.035043,0.020590,0.018153,0.018961
4,Weekend,0.030005,0.054143,0.024176,0.034679,0.023632,0.022105,0.051505,0.026343,0.026609,0.022041,0.021086,0.026744,0.028096,0.027275,0.025520,0.036115
29,sp3_6_meanvolume,0.029234,0.027853,0.027951,0.028922,0.027421,0.033172,0.027788,0.029979,0.035164,0.033927,0.026520,0.025272,0.028852,0.032721,0.027418,0.025546
0,marketIndexPrice,0.024364,0.027958,0.022388,0.020787,0.021378,0.019680,0.025107,0.027763,0.024891,0.026067,0.023896,0.022051,0.022342,0.028860,0.027666,0.024629


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
9,Month_COS,0.018049,0.000761,0.090657,0.029997,0.000763,0.003952,0.006180,0.004763,0.014002,0.018106,0.008845,0.009151,0.007702,0.005695,0.064790,0.005378
22,Solar_Generation,0.017556,0.016051,0.012823,0.017849,0.014455,0.016827,0.021066,0.020214,0.015895,0.018975,0.017308,0.014945,0.027694,0.017330,0.016058,0.015848
6,Weekday_SIN,0.017379,0.010206,0.030386,0.028480,0.018696,0.016754,0.026335,0.012722,0.012458,0.013815,0.017479,0.015174,0.022216,0.011765,0.012317,0.011884
23,WindOffshore_Generation,0.016237,0.018414,0.015789,0.015636,0.020675,0.015928,0.015364,0.015982,0.014520,0.015539,0.014110,0.016982,0.016476,0.016733,0.017015,0.014398
12,settPeriod_SIN,0.015562,0.021936,0.011225,0.018980,0.014979,0.019609,0.018725,0.012838,0.016143,0.013875,0.013623,0.017693,0.013105,0.012756,0.016898,0.011047
14,DANF_Demand,0.015009,0.012729,0.009435,0.011442,0.023910,0.010272,0.012009,0.013013,0.014306,0.013816,0.020811,0.010703,0.010568,0.019940,0.026045,0.016135
31,sp6_12_pavgprice,0.013580,0.012507,0.012219,0.012925,0.012523,0.013918,0.012098,0.013426,0.014144,0.013889,0.014599,0.010907,0.016319,0.015327,0.014729,0.014171
27,BrentOil_price,0.012791,0.013569,0.012933,0.014729,0.010910,0.015663,0.011262,0.011779,0.013761,0.014537,0.010407,0.010033,0.011622,0.013952,0.013352,0.013353
1,marketIndexVolume,0.012572,0.012997,0.010228,0.009717,0.012073,0.015746,0.013787,0.013414,0.010340,0.010475,0.014077,0.014466,0.012102,0.012618,0.012956,0.013588
33,sp6_12_totalvolume,0.011497,0.011502,0.010088,0.011087,0.011443,0.013215,0.010030,0.010663,0.010623,0.009193,0.010696,0.010929,0.011799,0.015954,0.011689,0.013537


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

In [ ]:
#RF_bid.predict(X_test)

In [ ]:
#df_modelAVG[(df_modelAVG['recordtype']=='BID')&(df_modelAVG['settlementDate']=='2023-06-10')&(df_modelAVG['settlementPeriod']==33)]

# OPTIMIZACIÓN INICIAL DE MODELOS (librería "Optuna")

### Carga Librerías

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
!pip install optuna

In [ ]:
import optuna

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
%matplotlib inline

In [ ]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

Busquemos los mejor hiperparámetros con Optuna

Para ello hay que envolver todo el modelo con la función objective

### XGBOOST (OFFER MODEL - SP)

In [ ]:
from matplotlib.cbook import print_cycles
unselect_vars = ['recordtype_label', 'Year_ordin']

select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

print(len(select_vars))

73


In [ ]:
recordtype = 'OFFER'

#test_dates = ['01','06','10','15','20','24','29']
test_dates = ['10','15','29']
#test_speriods = [14,24,34,42]
test_speriods = [14,34]

train_test = {}
cv_list = []

for x in test_dates:

  test_day = '2023-05-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}
    cv_list.append((train_indxs,test_indxs))

In [ ]:

def objective(trial, X, y, cv, scoring):
    rf_params = {
        "n_estimators": trial.suggest_int(name="n_estimators", low=25, high=150, log=False),
        "max_depth":trial.suggest_int(name="max_depth", low=4, high=8, log=False),
        "colsample_bytree":trial.suggest_float(name="colsample_bytree", low=0, high=1, log=False),
        "eta":trial.suggest_float(name="eta", low=0.2, high=0.4, log=False),
        "n_jobs": -1,
        "random_state": 0
        }

    steps = [('predictor', XGBRegressor(**rf_params))]

    pipe = Pipeline(steps)
    scores = cross_val_score(pipe, X, y, cv=cv,  scoring=scoring, n_jobs=-1)

    return scores.mean()


In [ ]:
#sklearn.metrics.get_scorer_names()

In [ ]:
%%time
from sklearn.model_selection import cross_val_score


#from sklearn.metrics import r2_score
#my_scorer = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
#sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))

# Create study that maximizes
study_XG1 = optuna.create_study(direction="maximize")


# Pass additional arguments inside another function
X_train = df_model_label[select_vars]
y_train = df_model_label['acceptedprice']

func = lambda trial: objective(trial, X_train, y_train, cv=cv_list,  scoring='neg_mean_squared_error')   #cv=kf


# Start optimizing with 100 trials
study_XG1.optimize(func, n_trials=25)

[I 2023-07-17 11:03:26,991] A new study created in memory with name: no-name-72ebda93-b767-4472-9554-7467a7ecdb2c
[I 2023-07-17 11:07:55,748] Trial 0 finished with value: -109458.12054810394 and parameters: {'n_estimators': 121, 'max_depth': 8, 'colsample_bytree': 0.37165136825677836, 'eta': 0.2569104525110394}. Best is trial 0 with value: -109458.12054810394.
[I 2023-07-17 11:09:34,063] Trial 1 finished with value: -450.17421643560846 and parameters: {'n_estimators': 97, 'max_depth': 5, 'colsample_bytree': 0.25675453961951, 'eta': 0.34065914503820927}. Best is trial 1 with value: -450.17421643560846.
[I 2023-07-17 11:10:28,388] Trial 2 finished with value: -659.4856770891125 and parameters: {'n_estimators': 141, 'max_depth': 4, 'colsample_bytree': 0.06393864712754072, 'eta': 0.2360545814399351}. Best is trial 1 with value: -450.17421643560846.
[I 2023-07-17 11:12:40,726] Trial 3 finished with value: -2092.459200834468 and parameters: {'n_estimators': 39, 'max_depth': 6, 'colsample_byt

CPU times: user 5min 21s, sys: 28.3 s, total: 5min 49s
Wall time: 1h 44min 49s


Visualizando el estudio

La verdad es que para obtener el mejor n_estimators tampoco era necesario usar optuna: un gridsearch corriente hubiera bastando...
Pero lo que es muy interesante es la visualización del estudio.

In [ ]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study_XG1)

In [ ]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study_XG1)

In [ ]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study_XG1)

In [ ]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study_XG1)

In [ ]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study_XG1)

Vamos a fija el número de árboles lo más bajo posible dentro de los mejores resultados. De este modo nos curamos del OF y no son demasiados para no reventar el tiempo de cálculo.

In [ ]:
''' XGBOOST



'''

' XGBOOST\n\n\n\n'

In [ ]:
# To get the dictionary of parameter name and parameter values:
print("Return a dictionary of parameter name and parameter values:",study_XG1.best_params)

# To get the best observed value of the objective function:
print("Return the best observed value of the objective function:",study_XG1.best_value)

# To get the best trial:
print("Return the best trial:",study_XG1.best_trial)

# To get all trials:
#print("Return all the trials:", study.trials)

Return a dictionary of parameter name and parameter values: {'n_estimators': 117, 'max_depth': 7, 'colsample_bytree': 0.8165565115010434, 'eta': 0.36542271508303015}
Return the best observed value of the objective function: -194.49406419484072
Return the best trial: FrozenTrial(number=15, state=TrialState.COMPLETE, values=[-194.49406419484072], datetime_start=datetime.datetime(2023, 7, 17, 11, 48, 15, 874010), datetime_complete=datetime.datetime(2023, 7, 17, 11, 54, 57, 560330), params={'n_estimators': 117, 'max_depth': 7, 'colsample_bytree': 0.8165565115010434, 'eta': 0.36542271508303015}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=150, log=False, low=25, step=1), 'max_depth': IntDistribution(high=8, log=False, low=4, step=1), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.0, step=None), 'eta': FloatDistribution(high=0.4, log=False, low=0.2, step=None)}, trial_id=15, value=None)


### XGBOOST (OFFER MODEL - SP PAVG. PRICE MODEL)

In [ ]:
from matplotlib.cbook import print_cycles
unselect_vars = ['recordtype_label', 'Year_ordin']

select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

print(len(select_vars))

39


In [ ]:
recordtype = 'OFFER'

#test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
test_dates = ['10','15','29']
#test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
test_speriods = [14,34]

train_test = {}
cv_list = []

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}
    cv_list.append((train_indxs,test_indxs))


In [ ]:

def objective(trial, X, y, cv, scoring):
    rf_params = {
        "n_estimators": trial.suggest_int(name="n_estimators", low=100, high=175, log=False),
        "max_depth":trial.suggest_int(name="max_depth", low=5, high=9, log=False),
        "colsample_bytree":trial.suggest_float(name="colsample_bytree", low=0.5, high=1, log=False),
        "eta":trial.suggest_float(name="eta", low=0.4, high=0.6, log=False),
        "n_jobs": -1,
        "random_state": 0
        }

    steps = [('predictor', XGBRegressor(**rf_params))]

    pipe = Pipeline(steps)
    scores = cross_val_score(pipe, X, y, cv=cv,  scoring=scoring, n_jobs=-1)

    return scores.mean()


In [ ]:
#sklearn.metrics.get_scorer_names()

In [ ]:
%%time
from sklearn.model_selection import cross_val_score


#from sklearn.metrics import r2_score
#my_scorer = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
#sqrt(mean_squared_error(y_train, RF_bid.predict(X_train)))

# Create study that minimizes
study_XG2 = optuna.create_study(direction="maximize")


# Pass additional arguments inside another function
X_train = df_modelAVG_label[select_vars]
y_train = df_modelAVG_label['sp_pavgprice']

func = lambda trial: objective(trial, X_train, y_train, cv=cv_list,  scoring='neg_mean_squared_error')   #cv=kf


# Start optimizing with 100 trials
study_XG2.optimize(func, n_trials=25)

[I 2023-07-17 15:12:30,179] A new study created in memory with name: no-name-1e7eef48-60ca-4780-8d12-3912d291e48c
[I 2023-07-17 15:15:31,399] Trial 0 finished with value: -481.5043898930118 and parameters: {'n_estimators': 162, 'max_depth': 5, 'colsample_bytree': 0.972289592368123, 'eta': 0.49919249155801415}. Best is trial 0 with value: -481.5043898930118.
[I 2023-07-17 15:17:58,345] Trial 1 finished with value: -341.4591164080851 and parameters: {'n_estimators': 112, 'max_depth': 7, 'colsample_bytree': 0.7431149374161137, 'eta': 0.4854809138416525}. Best is trial 1 with value: -341.4591164080851.
[I 2023-07-17 15:22:31,738] Trial 2 finished with value: -334.161174289876 and parameters: {'n_estimators': 173, 'max_depth': 8, 'colsample_bytree': 0.7746930345197622, 'eta': 0.4955254389880025}. Best is trial 2 with value: -334.161174289876.
[I 2023-07-17 15:25:28,024] Trial 3 finished with value: -362.27928807831694 and parameters: {'n_estimators': 168, 'max_depth': 5, 'colsample_bytree':

CPU times: user 3min 19s, sys: 13.6 s, total: 3min 33s
Wall time: 1h 32min 19s


Visualizando el estudio

La verdad es que para obtener el mejor n_estimators tampoco era necesario usar optuna: un gridsearch corriente hubiera bastando...
Pero lo que es muy interesante es la visualización del estudio.

In [ ]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study_XG2)

In [ ]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study_XG2)

In [ ]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study_XG2)

In [ ]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study_XG2)

In [ ]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study_XG2)

Vamos a fija el número de árboles lo más bajo posible dentro de los mejores resultados. De este modo nos curamos del OF y no son demasiados para no reventar el tiempo de cálculo.

In [ ]:
# To get the dictionary of parameter name and parameter values:
print("Return a dictionary of parameter name and parameter values:",study_XG2.best_params)

# To get the best observed value of the objective function:
print("Return the best observed value of the objective function:",study_XG2.best_value)

# To get the best trial:
print("Return the best trial:",study_XG2.best_trial)

# To get all trials:
#print("Return all the trials:", study.trials)

Return a dictionary of parameter name and parameter values: {'n_estimators': 144, 'max_depth': 8, 'colsample_bytree': 0.7948277932698775, 'eta': 0.5953352365063912}
Return the best observed value of the objective function: -221.94190680832298
Return the best trial: FrozenTrial(number=16, state=TrialState.COMPLETE, values=[-221.94190680832298], datetime_start=datetime.datetime(2023, 7, 17, 16, 14, 16, 50836), datetime_complete=datetime.datetime(2023, 7, 17, 16, 18, 9, 637754), params={'n_estimators': 144, 'max_depth': 8, 'colsample_bytree': 0.7948277932698775, 'eta': 0.5953352365063912}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=175, log=False, low=100, step=1), 'max_depth': IntDistribution(high=9, log=False, low=5, step=1), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'eta': FloatDistribution(high=0.6, log=False, low=0.4, step=None)}, trial_id=16, value=None)


# MODELOS AJUSTADOS (tras optimización)

**XGBOOST (BID/OFFER MODEL)**

Return a dictionary of parameter name and parameter values: {'n_estimators': 117, 'max_depth': 7, 'colsample_bytree': 0.8165565115010434, 'eta': 0.36542271508303015}
Return the best observed value of the objective function: -194.49406419484072

</br>

~~Return a dictionary of parameter name and parameter values: {'n_estimators': 200, 'max_depth': 5, 'colsample_bytree': 0.40328512411788453, 'eta': 0.49328006175341904}~~

~~Return the best observed value of the objective function: -166.94758914786445~~


**XGBOOST (SP PAVG. PRICE MODEL)**

Return a dictionary of parameter name and parameter values: {'n_estimators': 144, 'max_depth': 8, 'colsample_bytree': 0.7948277932698775, 'eta': 0.5953352365063912}
Return the best observed value of the objective function: -221.94190680832298

### XGBOOST (OFFER MODEL - SP)

In [ ]:
recordtype = 'OFFER'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]


train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

429303
3
429328
4
429357
8
429403
6
429420
13
429458
1
429560
19
429590
6
429626
4
430068
4
430092
4
430124
8
430191
11
430226
15
430312
10
430379
4
430406
3
430464
8
431516
2
431547
3
431580
8
431614
17
431652
4
431691
16
431751
22
431793
9
431836
7
432378
0
432380
0
432385
10
432487
12
432500
2
432507
2
432525
9
432545
1
432579
1
432884
4
432906
1
432958
10
433001
3
433015
14
433060
6
433102
6
433126
5
433172
11
433882
33
434080
32
434235
10
434328
47
434435
25
434572
29
434734
72
434922
32
435142
29
435803
4
435859
18
435921
5
435963
3
435971
8
435996
10
436095
7
436124
10
436185
0
436747
19
436825
2
436840
1
436843
0
436844
1
436869
1
436883
8
436943
17
437192
8
437530
6
437544
0
437561
7
437667
2
437673
2
437686
6
437714
1
437717
1
437764
2
437986
3
437997
1
438007
8
438041
6
438053
7
438060
2
438108
5
438127
1
438186
6
438980
9
439003
1
439029
2
439043
1
439045
0
439056
20
439160
23
439223
19
439368
14
439733
8
439753
6
439782
2
439797
1
439801
1
439842
21
439951
10
439970
11
440

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

In [ ]:
select_vars = []
data_columns = list(df_model_label.columns)
data_columns.remove('acceptedprice')
cat_vars = df_model_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

len(select_vars)

74

In [ ]:
#RF_bid = XGBRegressor(nthread=-1, verbosity=1, n_estimators=200, max_depth=5, colsample_bytree=0.40328512411788453, eta=0.49328006175341904)
RF_bid = XGBRegressor(nthread=-1, verbosity=1, n_estimators=117, max_depth=7, colsample_bytree=0.8165565115010434, eta=0.36542271508303015)

In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}



_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=429303 / RMSE=39.23272379499437 / MAE=18.386347100183794 / MeanReal: 218.188038663659 / MeanPred: 218.18563842773438
              Resid=1019.1604179850792 / AbsResidMean=18.386347100183794 / ResidMean=0.0023739885767825202 / StdResidMean=9.138702437012748e-05
Test scores: N=3 / RMSE=19.781570132107237 / MAE=19.144432067871094 / MeanReal: 146.33333333333334 / MeanPred: 127.18890380859375
             Resid=57.43329620361328 / AbsResidMean=19.144432067871094 / ResidMean=19.144432067871094 / StdResidMean=1.6600280349087173
1:01:42.568016
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=429328 / RMSE=38.37953786922209 / MAE=18.120420146796373 / MeanReal: 218.1832319634983 / MeanPred: 218.1785125732422
              Resid=2035.5881470834331 / AbsResidMean=18.120420146796373 / ResidMean=0.004741335638684605 / StdResidMean=8.93944

In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER MODEL OPT. - SP (scores MAPE).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_scores_df = pd.read_csv(file_path)
model_scores_df

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,429303,39.232724,18.386347,218.188039,218.188039,1019.160418,18.386347,0.002374,0.000091,3,19.781570,19.144432,146.333333,127.188904,57.433296,19.144432,19.144432,1.660028
1,2023-06-01,10,429328,38.379538,18.120420,218.183232,218.183232,2035.588147,18.120420,0.004741,0.000089,4,5.264220,4.592735,140.750000,139.527466,4.890137,4.592735,1.222534,1.280074
2,2023-06-01,16,429357,39.279366,18.304648,218.176753,218.176753,991.927166,18.304648,0.002310,0.000091,8,9.988768,9.834748,117.000000,122.092911,-40.743233,9.834748,-5.092904,1.074112
3,2023-06-01,22,429403,37.973290,18.159703,218.165789,218.165789,1651.778985,18.159703,0.003847,0.000088,6,6.567463,5.171604,116.166667,120.812416,-27.874504,5.171604,-4.645751,0.773674
4,2023-06-01,25,429420,38.000754,18.189728,218.161655,218.161655,1957.668619,18.189728,0.004559,0.000088,13,9.456625,7.062587,110.864615,105.237541,73.151888,7.062587,5.627068,0.584635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,443924,38.849407,17.910030,215.312814,215.312814,607.807229,17.910030,0.001369,0.000088,3,32.952480,19.809011,141.666667,121.857658,59.427032,19.809011,19.809011,8.777933
131,2023-06-29,31,443956,37.178059,17.638489,215.306597,215.306597,1867.388397,17.638489,0.004206,0.000084,3,10.438565,7.336764,134.333333,129.797104,13.608696,7.336764,4.536232,3.133798
132,2023-06-29,37,444001,38.478812,17.934151,215.298211,215.298211,1208.254913,17.934151,0.002721,0.000087,9,9.878234,7.112102,129.987778,125.847076,37.266381,7.112102,4.140709,0.996500
133,2023-06-29,40,444012,38.318452,17.861187,215.296154,215.296154,1946.876889,17.861187,0.004385,0.000086,1,5.418015,5.418015,142.500000,137.081985,5.418015,5.418015,5.418015,0.000000


In [ ]:
model_scores_df['rmse_test'].mean()

19.212782604555603

In [ ]:
model_scores_df['meanreal_test'].mean()

128.88886228080307

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER MODEL OPT. - SP (feat.importances).csv'
#model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.read_csv(file_path)
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,PC Status_label,0.254341,0.400253,0.321052,0.392893,0.377965,0.388497,0.403782,0.349089,0.354978,0.364637,0.453994,0.003291,0.000568,0.000625,0.001490,0.001997
1,Z6,0.181704,0.127367,0.145010,0.145236,0.162713,0.129623,0.144736,0.179180,0.154651,0.160268,0.102511,0.182624,0.272455,0.251055,0.305682,0.262449
2,sp3_6_pavgprice,0.065847,0.053186,0.061810,0.053515,0.058453,0.057456,0.053837,0.061041,0.067749,0.065849,0.058107,0.088462,0.076356,0.075307,0.076008,0.080568
3,LOC LAT,0.048346,0.041740,0.045005,0.035579,0.033785,0.035602,0.033927,0.045404,0.051782,0.052681,0.051077,0.075936,0.055897,0.053286,0.055109,0.058384
4,Coal_price,0.038307,0.035866,0.037816,0.030798,0.032609,0.030854,0.038353,0.033560,0.033528,0.035779,0.033900,0.054163,0.042236,0.043296,0.043129,0.048724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Weekend,0.001003,0.001955,0.000464,0.004540,0.000272,0.000174,0.000123,0.000283,0.000512,0.000359,0.000843,0.000513,0.002118,0.000076,0.002555,0.000264
69,Z4,0.000891,0.000419,0.000767,0.000888,0.000911,0.001370,0.000431,0.000563,0.000499,0.000512,0.001173,0.001176,0.001965,0.000561,0.001448,0.000682
70,Z2,0.000578,0.000109,0.000316,0.002360,0.000078,0.000306,0.000166,0.000226,0.000295,0.000345,0.000198,0.000254,0.003022,0.000246,0.000436,0.000316
71,Holiday,0.000253,0.000060,0.000317,0.000160,0.000219,0.000116,0.000301,0.000252,0.000242,0.000226,0.000217,0.000352,0.000231,0.000358,0.000248,0.000500


In [ ]:
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,PC Status_label,0.254341,0.400253,0.321052,0.392893,0.377965,0.388497,0.403782,0.349089,0.354978,0.364637,0.453994,0.003291,0.000568,0.000625,0.001490,0.001997
1,Z6,0.181704,0.127367,0.145010,0.145236,0.162713,0.129623,0.144736,0.179180,0.154651,0.160268,0.102511,0.182624,0.272455,0.251055,0.305682,0.262449
2,sp3_6_pavgprice,0.065847,0.053186,0.061810,0.053515,0.058453,0.057456,0.053837,0.061041,0.067749,0.065849,0.058107,0.088462,0.076356,0.075307,0.076008,0.080568
3,LOC LAT,0.048346,0.041740,0.045005,0.035579,0.033785,0.035602,0.033927,0.045404,0.051782,0.052681,0.051077,0.075936,0.055897,0.053286,0.055109,0.058384
4,Coal_price,0.038307,0.035866,0.037816,0.030798,0.032609,0.030854,0.038353,0.033560,0.033528,0.035779,0.033900,0.054163,0.042236,0.043296,0.043129,0.048724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Weekend,0.001003,0.001955,0.000464,0.004540,0.000272,0.000174,0.000123,0.000283,0.000512,0.000359,0.000843,0.000513,0.002118,0.000076,0.002555,0.000264
69,Z4,0.000891,0.000419,0.000767,0.000888,0.000911,0.001370,0.000431,0.000563,0.000499,0.000512,0.001173,0.001176,0.001965,0.000561,0.001448,0.000682
70,Z2,0.000578,0.000109,0.000316,0.002360,0.000078,0.000306,0.000166,0.000226,0.000295,0.000345,0.000198,0.000254,0.003022,0.000246,0.000436,0.000316
71,Holiday,0.000253,0.000060,0.000317,0.000160,0.000219,0.000116,0.000301,0.000252,0.000242,0.000226,0.000217,0.000352,0.000231,0.000358,0.000248,0.000500


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,PC Status_label,0.254341,0.400253,0.321052,0.392893,0.377965,0.388497,0.403782,0.349089,0.354978,0.364637,0.453994,0.003291,0.000568,0.000625,0.001490,0.001997
1,Z6,0.181704,0.127367,0.145010,0.145236,0.162713,0.129623,0.144736,0.179180,0.154651,0.160268,0.102511,0.182624,0.272455,0.251055,0.305682,0.262449
2,sp3_6_pavgprice,0.065847,0.053186,0.061810,0.053515,0.058453,0.057456,0.053837,0.061041,0.067749,0.065849,0.058107,0.088462,0.076356,0.075307,0.076008,0.080568
3,LOC LAT,0.048346,0.041740,0.045005,0.035579,0.033785,0.035602,0.033927,0.045404,0.051782,0.052681,0.051077,0.075936,0.055897,0.053286,0.055109,0.058384
4,Coal_price,0.038307,0.035866,0.037816,0.030798,0.032609,0.030854,0.038353,0.033560,0.033528,0.035779,0.033900,0.054163,0.042236,0.043296,0.043129,0.048724
5,Solar_Generation,0.037526,0.028461,0.027295,0.031665,0.030300,0.023546,0.025966,0.032658,0.033897,0.033202,0.030318,0.050155,0.060677,0.053856,0.049207,0.051686
6,BMU Type_label,0.030684,0.000609,0.001362,0.001334,0.001041,0.000895,0.003058,0.002728,0.001871,0.001820,0.001472,0.004370,0.097910,0.135987,0.088722,0.117075
7,BZONE GENERATION,0.024320,0.019426,0.023849,0.016891,0.019548,0.020285,0.017410,0.023455,0.018969,0.024730,0.017871,0.040126,0.033828,0.028660,0.036025,0.023734
8,settPeriod_COS,0.023814,0.021044,0.021906,0.019286,0.019978,0.021035,0.018542,0.022760,0.022052,0.021133,0.020742,0.031615,0.029381,0.027814,0.026677,0.033246
9,acceptedvolume,0.017788,0.016249,0.017841,0.016219,0.016789,0.015567,0.017472,0.015886,0.014578,0.014723,0.014216,0.021244,0.022743,0.022087,0.021261,0.019951


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
53,LOC Center LONG,0.002165,0.001144,0.000965,0.001775,0.000647,0.001743,0.001524,0.002423,0.007258,0.001350,0.002455,0.004655,0.000868,0.001854,0.000884,0.002936
54,SystemTotalLoad_Consumption,0.002165,0.000999,0.001477,0.001308,0.001215,0.001354,0.001120,0.001885,0.001838,0.001707,0.001390,0.005167,0.003548,0.002767,0.002649,0.004052
55,DAIG_spnGeneration,0.002071,0.002093,0.001670,0.002113,0.001669,0.001467,0.002690,0.002205,0.002101,0.001767,0.001790,0.003429,0.001864,0.001684,0.002226,0.002293
56,GenerationForecast_Production,0.002047,0.000772,0.001344,0.001277,0.002228,0.002130,0.002395,0.002804,0.002787,0.002584,0.002424,0.002314,0.001632,0.001847,0.002231,0.001938
57,Z7,0.002033,0.000647,0.004466,0.001693,0.001888,0.002187,0.002091,0.007510,0.000267,0.000480,0.001362,0.001938,0.000555,0.000775,0.000233,0.004398
58,Z11,0.002021,0.001653,0.003100,0.001650,0.002254,0.002469,0.001984,0.001222,0.000970,0.001286,0.000921,0.001272,0.003020,0.003375,0.002355,0.002791
59,Transmission Loss Factor,0.002019,0.001308,0.002119,0.001240,0.002650,0.001315,0.002312,0.003821,0.001139,0.001321,0.002504,0.004128,0.001370,0.002329,0.001457,0.001272
60,sp6_12_totalvolume,0.001960,0.001778,0.001391,0.003185,0.001060,0.001756,0.002583,0.001286,0.001323,0.001643,0.001096,0.002182,0.002841,0.002382,0.002698,0.002193
61,marketIndexVolume,0.001890,0.001590,0.002205,0.001775,0.001647,0.002123,0.001796,0.002123,0.001680,0.001601,0.002148,0.002091,0.001816,0.001714,0.002115,0.001928
62,DATF_Demand,0.001791,0.002768,0.002407,0.001710,0.001715,0.001650,0.002018,0.001339,0.001839,0.001866,0.002472,0.002286,0.001290,0.001608,0.001062,0.000830


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

### XGBOOST (OFFER MODEL - SP PAVG. PRICE MODEL)

In [ ]:
recordtype = 'OFFER'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
for x in test_dates:
  for y in test_speriods:

    test_day = '2023-06-' + x
    test_sp = y

    #print(test_day)
    #print(test_sp)
    print(len(train_test[test_day][test_sp]['train_indxs']))
    print(len(train_test[test_day][test_sp]['test_indxs']))

223746
3
223769
3
223793
6
223828
5
223842
7
223866
1
223919
10
223939
5
223974
4
224232
3
224251
3
224277
6
224324
7
224347
9
224397
6
224446
4
224467
3
224508
7
224942
2
224967
3
224994
5
225016
9
225039
4
225061
5
225094
11
225116
7
225150
6
225428
0
225430
0
225435
5
225483
5
225489
2
225496
2
225510
6
225527
1
225558
1
225740
4
225759
1
225788
7
225816
2
225828
10
225861
5
225883
4
225900
3
225929
7
226321
13
226397
12
226461
8
226504
13
226536
8
226593
9
226644
13
226683
13
226763
10
227105
4
227138
11
227176
4
227204
3
227211
5
227231
6
227286
6
227308
8
227354
0
227608
11
227653
2
227666
1
227669
0
227670
1
227687
1
227697
4
227722
8
227809
5
228008
5
228021
0
228033
5
228076
1
228081
2
228093
6
228116
1
228119
1
228146
1
228308
3
228319
1
228328
6
228348
4
228356
4
228360
2
228388
4
228400
1
228443
4
228763
5
228782
1
228800
1
228810
1
228812
0
228822
8
228861
10
228888
10
228956
9
229190
4
229206
4
229227
2
229240
1
229244
1
229266
9
229321
8
229336
6
229364
3
229656
8
229715

In [ ]:
unselect_vars = ['recordtype_label', 'Year_ordin']

In [ ]:
select_vars = []
data_columns = list(df_modelAVG_label.columns)
data_columns.remove('sp_pavgprice')
cat_vars = df_modelAVG_label.select_dtypes(exclude=["number"]).columns

for x in data_columns:
  if (x not in cat_vars) & (x not in unselect_vars):
    select_vars.append(x)
  else:
    next

len(select_vars)

40

In [ ]:
RF_bid = XGBRegressor(nthread=-1, verbosity=1, n_estimators=144, max_depth=8, colsample_bytree=0.7948277932698775, eta=0.5953352365063912)

In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())
#test_dates = ['2023-06-01']
#test_speriods = [22]


model_scores = {}
model_importances = {'feature':select_vars}


test_days = []
test_sps = []

size_trains = []
size_tests = []
rmse_trains = []
rmse_tests = []
mae_trains = []
mae_tests = []
#mape_trains = []
#mape_tests = []
meanreal_trains = []
meanreal_tests = []
meanpred_trains = []
meanpred_tests = []

resid_trains = []
resid_tests = []
absresidmean_trains = []
absresidmean_tests = []
residmean_trains = []
residmean_tests = []
stdresid_trains = []
stdresid_tests = []

#r2_trains = []
#r2_tests = []



for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], select_vars]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'], select_vars]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    RF_bid.fit(X_train, y_train)


    size_train = y_train.shape[0]
    y_pred = RF_bid.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train, y_pred))
    mae_train = mean_absolute_error(y_train, y_pred)
    #mape_train = mean_absolute_percentage_error(y_train, y_pred)
    meanreal_train = y_train.mean()
    meanpred_train = y_pred.mean()
    resid_train = sum(y_train - y_pred)
    absresidmean_train = np.mean(abs(y_train - y_pred))
    residmean_train = np.mean(y_train - y_pred)
    stdresid_train = np.std(y_train - y_pred) / size_train
    #r2_train = r2_score(y_train, RF_bid.predict(X_train))
    print(f'Train scores: N={size_train} / RMSE={rmse_train} / MAE={mae_train} / MeanReal: {meanreal_train} / MeanPred: {meanpred_train}')
    print(f'              Resid={resid_train} / AbsResidMean={absresidmean_train} / ResidMean={residmean_train} / StdResidMean={stdresid_train}')

    size_test = y_test.shape[0]
    if size_test > 0:
      y_pred = RF_bid.predict(X_test)
      rmse_test = sqrt(mean_squared_error(y_test, y_pred))
      mae_test = mean_absolute_error(y_test, y_pred)
      #mape_test = mean_absolute_percentage_error(y_test, y_pred)
      meanreal_test = y_test.mean()
      meanpred_test = y_pred.mean()
      resid_test = sum(y_test - y_pred)
      absresidmean_test = np.mean(abs(y_test - y_pred))
      residmean_test = np.mean(y_test - y_pred)
      stdresid_test = np.std(y_test - y_pred) / size_test
      #r2_test = r2_score(y_test, RF_bid.predict(X_test))
      print(f'Test scores: N={size_test} / RMSE={rmse_test} / MAE={mae_test} / MeanReal: {meanreal_test} / MeanPred: {meanpred_test}')
      print(f'             Resid={resid_test} / AbsResidMean={absresidmean_test} / ResidMean={residmean_test} / StdResidMean={stdresid_test}')
    else:
      rmse_test = np.nan
      mae_test = np.nan
      #mape_test = np.nan
      meanreal_test = np.nan
      meanpred_test = np.nan
      resid_test = np.nan
      absresidmean_test = np.nan
      residmean_test = np.nan
      stdresid_test = np.nan
      #r2_test = np.nan


    size_trains.append(size_train)
    size_tests.append(size_test)
    rmse_trains.append(rmse_train)
    rmse_tests.append(rmse_test)
    mae_trains.append(mae_train)
    mae_tests.append(mae_test)
    #mape_trains.append(mape_train)
    #mape_tests.append(mape_test)
    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)
    meanpred_trains.append(meanreal_train)
    meanpred_tests.append(meanpred_test)

    resid_trains.append(resid_train)
    resid_tests.append(resid_test)
    absresidmean_trains.append(absresidmean_train)
    absresidmean_tests.append(absresidmean_test)
    residmean_trains.append(residmean_train)
    residmean_tests.append(residmean_test)
    stdresid_trains.append(stdresid_train)
    stdresid_tests.append(stdresid_test)

    #r2_trains.append(r2_train)
    #r2_tests.append(r2_test)


    model_imp = 'imp_' + x
    model_importances[model_imp] = RF_bid.feature_importances_


    end = time.time()
    print(timedelta(minutes=end-start))
    #print('\n')

model_scores = {'test_day':test_days,'test_sp':test_sps,

                'size_train':size_trains,'rmse_train':rmse_trains,'mae_train':mae_trains,'meanreal_train':meanreal_trains,'meanpred_train':meanpred_trains,
                'resid_train':resid_trains,'absresidmean_train':absresidmean_trains,'residmean_train':residmean_trains,'stdresid_train':stdresid_trains,

                'size_test':size_tests,'rmse_test':rmse_tests,'mae_test':mae_tests,'meanreal_test':meanreal_tests,'meanpred_test':meanpred_tests,
                'resid_test':resid_tests,'absresidmean_test':absresidmean_tests,'residmean_test':residmean_tests,'stdresid_test':stdresid_tests}



_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 4)
Train scores: N=223746 / RMSE=26.57575725921939 / MAE=16.22724544252102 / MeanReal: 216.01129027849322 / MeanPred: 216.009765625
              Resid=344.42403941619415 / AbsResidMean=16.22724544252102 / ResidMean=0.001539352834989768 / StdResidMean=0.00011877645729817416
Test scores: N=3 / RMSE=2.704164943888495 / MAE=2.4693450927734375 / MeanReal: 146.33333333333334 / MeanPred: 146.5199737548828
             Resid=-0.5599517822265625 / AbsResidMean=2.4693450927734375 / ResidMean=-0.18665059407552084 / StdResidMean=0.8992385420446148
0:38:30.229568
_____________________________________________
Model XGBoost (Test: Day 2023-06-01 / SP 10)
Train scores: N=223769 / RMSE=28.457094498556994 / MAE=16.865865866180826 / MeanReal: 216.00326002261912 / MeanPred: 216.00350952148438
              Resid=-60.08323849081141 / AbsResidMean=16.865865866180826 / ResidMean=-0.00026850563970086074 / StdResidMean=0.00

In [ ]:
pd.DataFrame(model_scores).round(4)

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,223746,26.5758,16.2272,216.0113,216.0113,344.4240,16.2272,0.0015,0.0001,3,2.7042,2.4693,146.3333,146.5200,-0.5600,2.4693,-0.1867,0.8992
1,2023-06-01,10,223769,28.4571,16.8659,216.0033,216.0033,-60.0832,16.8659,-0.0003,0.0001,3,4.6263,4.4964,146.3230,150.8194,-13.4892,4.4964,-4.4964,0.3628
2,2023-06-01,16,223793,27.9012,16.7265,215.9935,215.9935,47.0565,16.7265,0.0002,0.0001,6,13.3272,12.2282,113.9314,110.2342,22.1828,12.2282,3.6971,2.1340
3,2023-06-01,22,223828,26.7501,16.2644,215.9775,215.9775,-183.6487,16.2644,-0.0008,0.0001,5,5.1509,4.7261,114.4000,111.7306,13.3471,4.7261,2.6694,0.8810
4,2023-06-01,25,223842,26.9185,16.2487,215.9711,215.9711,217.3830,16.2487,0.0010,0.0001,7,8.5184,6.5922,110.7397,114.3288,-25.1239,6.5922,-3.5891,1.1036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,230984,28.0870,16.7542,213.3094,213.3094,-56.7038,16.7542,-0.0002,0.0001,2,7.8747,6.6673,112.5036,116.6938,-8.3803,6.6673,-4.1902,3.3336
131,2023-06-29,31,231007,27.0733,16.2889,213.3008,213.3008,-140.8887,16.2889,-0.0006,0.0001,3,22.6988,19.8931,134.3333,117.4481,50.6556,19.8931,16.8852,5.0566
132,2023-06-29,37,231038,27.1010,16.3210,213.2902,213.2902,129.9708,16.3210,0.0006,0.0001,7,15.0602,9.5023,132.0964,130.7776,9.2315,9.5023,1.3188,2.1432
133,2023-06-29,40,231047,26.7694,16.1888,213.2871,213.2871,497.6283,16.1888,0.0022,0.0001,1,2.5083,2.5083,142.5000,139.9917,2.5083,2.5083,2.5083,0.0000


In [ ]:
model_scores_df = pd.DataFrame(model_scores)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER SP PAVG. PRICE MODEL OPT. - SP (scores MAPE).csv'
model_scores_df.to_csv(file_path, index=False)

In [ ]:
model_scores_df = pd.read_csv(file_path)
model_scores_df

,test_day,test_sp,size_train,rmse_train,mae_train,meanreal_train,meanpred_train,resid_train,absresidmean_train,residmean_train,stdresid_train,size_test,rmse_test,mae_test,meanreal_test,meanpred_test,resid_test,absresidmean_test,residmean_test,stdresid_test
0,2023-06-01,4,223746,26.575757,16.227245,216.011290,216.011290,344.424039,16.227245,0.001539,0.000119,3,2.704165,2.469345,146.333333,146.519974,-0.559952,2.469345,-0.186651,0.899239
1,2023-06-01,10,223769,28.457094,16.865866,216.003260,216.003260,-60.083238,16.865866,-0.000269,0.000127,3,4.626284,4.496404,146.322998,150.819412,-13.489212,4.496404,-4.496404,0.362837
2,2023-06-01,16,223793,27.901152,16.726513,215.993548,215.993548,47.056540,16.726513,0.000210,0.000125,6,13.327198,12.228230,113.931373,110.234245,22.182779,12.228230,3.697130,2.134020
3,2023-06-01,22,223828,26.750102,16.264366,215.977485,215.977485,-183.648713,16.264366,-0.000820,0.000120,5,5.150912,4.726138,114.400000,111.730576,13.347137,4.726138,2.669427,0.881046
4,2023-06-01,25,223842,26.918465,16.248676,215.971058,215.971058,217.383033,16.248676,0.000971,0.000120,7,8.518440,6.592213,110.739663,114.328789,-25.123913,6.592213,-3.589130,1.103630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023-06-29,25,230984,28.086981,16.754218,213.309403,213.309403,-56.703844,16.754218,-0.000245,0.000122,2,7.874669,6.667299,112.503619,116.693787,-8.380343,6.667299,-4.190172,3.333649
131,2023-06-29,31,231007,27.073318,16.288879,213.300785,213.300785,-140.888701,16.288879,-0.000610,0.000117,3,22.698818,19.893084,134.333333,117.448143,50.655571,19.893084,16.885190,5.056642
132,2023-06-29,37,231038,27.101011,16.320967,213.290190,213.290190,129.970774,16.320967,0.000563,0.000117,7,15.060214,9.502278,132.096364,130.777588,9.231453,9.502278,1.318779,2.143195
133,2023-06-29,40,231047,26.769421,16.188783,213.287117,213.287117,497.628278,16.188783,0.002154,0.000116,1,2.508316,2.508316,142.500000,139.991684,2.508316,2.508316,2.508316,0.000000


In [ ]:
model_scores_df['rmse_test'].mean()

14.8012699671404

In [ ]:
model_scores_df['meanreal_test'].mean()

128.44660073881215

In [ ]:
model_imp_df = pd.DataFrame(model_importances)

col_list= list(model_imp_df.columns)
col_list.remove('feature')

model_imp_df['imp_sum'] = model_imp_df[col_list].mean(axis=1)
model_imp_df.insert(loc=1, column='imp_sum', value=model_imp_df.pop('imp_sum'))
model_imp_df = model_imp_df.sort_values(by='imp_sum', ascending=False)

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Results/XGBoost/XGBoost OFFER SP PAVG. PRICE MODEL OPT. - SP (feat.importances).csv'
#model_imp_df.to_csv(file_path, index=False)

In [ ]:
model_imp_df = pd.read_csv(file_path)
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.219424,0.220154,0.221031,0.213738,0.210362,0.222195,0.223468,0.209505,0.211862,0.220775,0.207273,0.218680,0.218434,0.225941,0.238024,0.229914
1,settPeriod_COS,0.154967,0.159906,0.171425,0.165679,0.146860,0.160823,0.156499,0.149962,0.146327,0.162844,0.158781,0.160181,0.141166,0.150519,0.147656,0.145881
2,Day_ordin,0.066664,0.067780,0.062328,0.056367,0.066841,0.068460,0.076781,0.063096,0.057894,0.063378,0.064561,0.073075,0.069973,0.059015,0.065017,0.085399
3,Coal_price,0.051583,0.042114,0.044233,0.041889,0.051500,0.041219,0.042874,0.078139,0.082431,0.075466,0.066091,0.043844,0.044907,0.040380,0.035007,0.043644
4,LOC Center LONG,0.045762,0.046205,0.048129,0.047326,0.043097,0.045218,0.047239,0.044015,0.045768,0.048636,0.047973,0.043122,0.044465,0.044587,0.045583,0.045063
5,sp3_6_meanvolume,0.032955,0.033400,0.033998,0.031577,0.034328,0.034082,0.030729,0.029873,0.035072,0.031503,0.032304,0.030567,0.033771,0.036707,0.031232,0.035188
6,DAMargin,0.028837,0.027459,0.031771,0.029280,0.027921,0.031070,0.028827,0.026062,0.028484,0.027903,0.030952,0.027301,0.033327,0.029499,0.026478,0.026214
7,WindOffshore_Generation,0.022327,0.021325,0.025011,0.020552,0.018010,0.018915,0.026476,0.018893,0.017598,0.023032,0.022027,0.017443,0.026751,0.022731,0.023240,0.032905
8,settlementDate_ordin,0.018843,0.017828,0.019334,0.019173,0.018374,0.021360,0.023657,0.016901,0.018187,0.018205,0.016128,0.016635,0.020142,0.020123,0.020024,0.016569
9,LOC Center LAT,0.018816,0.027241,0.014381,0.019553,0.014079,0.019549,0.015164,0.017621,0.017824,0.018706,0.014043,0.030211,0.015669,0.019850,0.020399,0.017945


In [ ]:
model_imp_df

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.219424,0.220154,0.221031,0.213738,0.210362,0.222195,0.223468,0.209505,0.211862,0.220775,0.207273,0.218680,0.218434,0.225941,0.238024,0.229914
1,settPeriod_COS,0.154967,0.159906,0.171425,0.165679,0.146860,0.160823,0.156499,0.149962,0.146327,0.162844,0.158781,0.160181,0.141166,0.150519,0.147656,0.145881
2,Day_ordin,0.066664,0.067780,0.062328,0.056367,0.066841,0.068460,0.076781,0.063096,0.057894,0.063378,0.064561,0.073075,0.069973,0.059015,0.065017,0.085399
3,Coal_price,0.051583,0.042114,0.044233,0.041889,0.051500,0.041219,0.042874,0.078139,0.082431,0.075466,0.066091,0.043844,0.044907,0.040380,0.035007,0.043644
4,LOC Center LONG,0.045762,0.046205,0.048129,0.047326,0.043097,0.045218,0.047239,0.044015,0.045768,0.048636,0.047973,0.043122,0.044465,0.044587,0.045583,0.045063
5,sp3_6_meanvolume,0.032955,0.033400,0.033998,0.031577,0.034328,0.034082,0.030729,0.029873,0.035072,0.031503,0.032304,0.030567,0.033771,0.036707,0.031232,0.035188
6,DAMargin,0.028837,0.027459,0.031771,0.029280,0.027921,0.031070,0.028827,0.026062,0.028484,0.027903,0.030952,0.027301,0.033327,0.029499,0.026478,0.026214
7,WindOffshore_Generation,0.022327,0.021325,0.025011,0.020552,0.018010,0.018915,0.026476,0.018893,0.017598,0.023032,0.022027,0.017443,0.026751,0.022731,0.023240,0.032905
8,settlementDate_ordin,0.018843,0.017828,0.019334,0.019173,0.018374,0.021360,0.023657,0.016901,0.018187,0.018205,0.016128,0.016635,0.020142,0.020123,0.020024,0.016569
9,LOC Center LAT,0.018816,0.027241,0.014381,0.019553,0.014079,0.019549,0.015164,0.017621,0.017824,0.018706,0.014043,0.030211,0.015669,0.019850,0.020399,0.017945


In [ ]:
model_imp_df.head(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
0,sp3_6_pavgprice,0.219424,0.220154,0.221031,0.213738,0.210362,0.222195,0.223468,0.209505,0.211862,0.220775,0.207273,0.218680,0.218434,0.225941,0.238024,0.229914
1,settPeriod_COS,0.154967,0.159906,0.171425,0.165679,0.146860,0.160823,0.156499,0.149962,0.146327,0.162844,0.158781,0.160181,0.141166,0.150519,0.147656,0.145881
2,Day_ordin,0.066664,0.067780,0.062328,0.056367,0.066841,0.068460,0.076781,0.063096,0.057894,0.063378,0.064561,0.073075,0.069973,0.059015,0.065017,0.085399
3,Coal_price,0.051583,0.042114,0.044233,0.041889,0.051500,0.041219,0.042874,0.078139,0.082431,0.075466,0.066091,0.043844,0.044907,0.040380,0.035007,0.043644
4,LOC Center LONG,0.045762,0.046205,0.048129,0.047326,0.043097,0.045218,0.047239,0.044015,0.045768,0.048636,0.047973,0.043122,0.044465,0.044587,0.045583,0.045063
5,sp3_6_meanvolume,0.032955,0.033400,0.033998,0.031577,0.034328,0.034082,0.030729,0.029873,0.035072,0.031503,0.032304,0.030567,0.033771,0.036707,0.031232,0.035188
6,DAMargin,0.028837,0.027459,0.031771,0.029280,0.027921,0.031070,0.028827,0.026062,0.028484,0.027903,0.030952,0.027301,0.033327,0.029499,0.026478,0.026214
7,WindOffshore_Generation,0.022327,0.021325,0.025011,0.020552,0.018010,0.018915,0.026476,0.018893,0.017598,0.023032,0.022027,0.017443,0.026751,0.022731,0.023240,0.032905
8,settlementDate_ordin,0.018843,0.017828,0.019334,0.019173,0.018374,0.021360,0.023657,0.016901,0.018187,0.018205,0.016128,0.016635,0.020142,0.020123,0.020024,0.016569
9,LOC Center LAT,0.018816,0.027241,0.014381,0.019553,0.014079,0.019549,0.015164,0.017621,0.017824,0.018706,0.014043,0.030211,0.015669,0.019850,0.020399,0.017945


In [ ]:
model_imp_df.tail(20)

,feature,imp_sum,imp_2023-06-01,imp_2023-06-03,imp_2023-06-05,imp_2023-06-07,imp_2023-06-09,imp_2023-06-11,imp_2023-06-13,imp_2023-06-15,imp_2023-06-17,imp_2023-06-19,imp_2023-06-21,imp_2023-06-23,imp_2023-06-25,imp_2023-06-27,imp_2023-06-29
19,sp6_12_meanvolume,0.013568,0.008978,0.009999,0.015769,0.010229,0.015502,0.009246,0.011954,0.013951,0.018266,0.012544,0.011103,0.017535,0.017373,0.014923,0.016150
20,BrentOil_price,0.013386,0.025102,0.011703,0.013232,0.013226,0.016049,0.010997,0.011349,0.012887,0.009773,0.011574,0.013610,0.011329,0.014274,0.011734,0.013946
21,BMU GSP Group Name_label,0.013292,0.012535,0.011882,0.013158,0.012311,0.012464,0.012657,0.016399,0.012561,0.012888,0.019524,0.012738,0.011754,0.012944,0.013196,0.012373
22,settPeriod_SIN,0.012633,0.013044,0.011150,0.014080,0.011280,0.016076,0.015479,0.011330,0.010695,0.012455,0.012710,0.010087,0.011610,0.011707,0.012893,0.014901
23,Solar_Generation,0.012217,0.013528,0.013206,0.015543,0.013652,0.014182,0.011775,0.011724,0.011871,0.010156,0.008717,0.010387,0.012223,0.013582,0.010895,0.011813
24,Day_COS,0.012054,0.012454,0.011130,0.014610,0.009362,0.013837,0.018095,0.011271,0.011651,0.011029,0.011611,0.008981,0.010428,0.010783,0.012894,0.012669
25,Day_SIN,0.011885,0.013187,0.011444,0.010510,0.011300,0.010714,0.010563,0.010921,0.010444,0.009658,0.015566,0.014314,0.011324,0.011932,0.014557,0.011837
26,GenerationForecast_Production,0.011830,0.008096,0.008997,0.006729,0.021494,0.008532,0.009163,0.011139,0.008051,0.012642,0.021539,0.007781,0.011359,0.012756,0.020146,0.009023
27,DAID_spnDemand,0.011390,0.008273,0.022041,0.011924,0.007754,0.019001,0.012477,0.008150,0.017487,0.010503,0.009802,0.007371,0.008703,0.008907,0.010039,0.008420
28,sp6_12_pavgprice,0.010808,0.010505,0.011646,0.010115,0.009022,0.010347,0.010286,0.011122,0.011115,0.012519,0.010188,0.010058,0.013347,0.010071,0.012753,0.009022


In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].head(20)

In [ ]:
#model_imp_df[~model_imp_df['feature'].str.contains('sp')].tail(20)

In [ ]:
#RF_bid.predict(X_test)

In [ ]:
#df_modelAVG[(df_modelAVG['recordtype']=='BID')&(df_modelAVG['settlementDate']=='2023-06-10')&(df_modelAVG['settlementPeriod']==33)]

# Varios

### XGBOOST (BID/OFFER MODEL) - Modelo OFFER (Predicción por SP)

In [100]:
recordtype = 'OFFER'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']<test_day)].copy()
    train_day = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_model_label[(df_model_label['recordtype']==recordtype)&(df_model_label['settlementDate']==test_day)&(df_model_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}

test_days = []
test_sps = []

meanreal_trains = []
meanreal_tests = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_model_label.loc[train_test[x][y]['train_indxs'],:]
    y_train = df_model_label.loc[train_test[x][y]['train_indxs'], 'acceptedprice']

    X_test = df_model_label.loc[train_test[x][y]['test_indxs'],:]
    y_test = df_model_label.loc[train_test[x][y]['test_indxs'],'acceptedprice']


    meanreal_train = y_train.mean()

    size_test = y_test.shape[0]
    if size_test > 0:
      meanreal_test = y_test.mean()
    else:
      meanreal_test = np.nan

    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)


model_scores = {'test_day':test_days,'test_sp':test_sps,
                'meanreal_train':meanreal_trains,
                'meanreal_test':meanreal_tests}


In [102]:
model_scores_df = pd.DataFrame(model_scores)

In [103]:
model_scores_df['meanreal_test'].mean()

128.88886228080307

### XGBOOST (SP PAVG. PRICE MODEL) - Modelo OFFER (Predicción por SP)

In [104]:
recordtype = 'OFFER'

test_dates = ['01','03','05','07','09','11','13','15','17','19','21','23','25','27','29']
#test_dates = ['01','06','10','15','20','24','29']
#test_dates = ['10','15','29']
test_speriods = [4,10,16,22,25,31,37,40,48]
#test_speriods = [14,24,34,42]
#test_speriods = [14,34]

train_test = {}

for x in test_dates:

  test_day = '2023-06-' + x
  train_test[test_day] = {}

  for y in test_speriods:

    test_sp = y


    train_prev = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']<test_day)].copy()
    train_day = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']<test_sp)].copy()
    train = pd.concat([train_prev, train_day])
    train_indxs = list(train.index.values)

    test = df_modelAVG_label[(df_modelAVG_label['recordtype']==recordtype)&(df_modelAVG_label['settlementDate']==test_day)&(df_modelAVG_label['settlementPeriod']==test_sp)].copy()
    test_indxs = list(test.index.values)

    train_test[test_day][test_sp] = {'train_indxs':train_indxs,'test_indxs':test_indxs}


In [ ]:
recordtype = 'OFFER'
test_dates = list(train_test.keys())
test_speriods = list(train_test[test_dates[0]].keys())

model_scores = {}

test_days = []
test_sps = []

meanreal_trains = []
meanreal_tests = []


for x in test_dates:
  for y in test_speriods:

    test_days.append(x)
    test_sps.append(y)

    print('_'*45)
    print(f'Model XGBoost (Test: Day {x} / SP {y})')

    start = time.time()

    X_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'],:]
    y_train = df_modelAVG_label.loc[train_test[x][y]['train_indxs'], 'sp_pavgprice']

    X_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],:]
    y_test = df_modelAVG_label.loc[train_test[x][y]['test_indxs'],'sp_pavgprice']


    meanreal_train = y_train.mean()

    size_test = y_test.shape[0]
    if size_test > 0:
      meanreal_test = y_test.mean()
    else:
      meanreal_test = np.nan

    meanreal_trains.append(meanreal_train)
    meanreal_tests.append(meanreal_test)


model_scores = {'test_day':test_days,'test_sp':test_sps,
                'meanreal_train':meanreal_trains,
                'meanreal_test':meanreal_tests}


In [106]:
model_scores_df = pd.DataFrame(model_scores)

In [107]:
model_scores_df['meanreal_test'].mean()

128.44660073881215